# The python program of anonymizing address data(string data)

## importing libraries

In [2]:
#importing libralies that are needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pykakasi import kakasi
import mojimoji
import struct
from collections import Counter
import time
import copy

## defining the function

In [3]:
#defining the functions that will be used in this program

#changing the numbers into double precision
def binary(num):
    return ''.join(bin(c).replace('0b', '').rjust(8, '0') for c in struct.pack('!d', num))

#changing the double float in to a number
def float_to_num(target):
    for i in range(0,len(target),8):
        if i == 0:
            tmp = struct.pack("!B",int(target[i:i+8],2))
        else:
            tmp = tmp + struct.pack("!B",int(target[i:i+8],2))
    return np.float64(struct.unpack("!d",tmp)[0])

## Making the data 

In [4]:
#initial coding to convert the japanese character into a latin alphabet
kakasi = kakasi()
kakasi.setMode('H', 'a')
kakasi.setMode('K', 'a')
kakasi.setMode('J', 'a')
conv = kakasi.getConverter()

In [6]:
#Gettng the data from the csv file and deleting the columns that are not needed
data_address_company_original = pd.read_csv("/home/harada/Documents/data/address_data/14_kanagawa_all_20180928.csv",encoding="SHIFT-JIS", header=None)
columns_list = [0,1,2,3,4,5,7,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
data_address_company_original = data_address_company_original.drop(columns=columns_list)
data_address_company_original.columns = ["company_name","company_group_id","address_1","address_2","address_3"]
data_address_company_original

,company_name,company_group_id,address_1,address_2,address_3
0,藤沢簡易裁判所,101,神奈川県,藤沢市,朝日町１−８
1,神奈川県,201,神奈川県,横浜市中区,日本大通１
2,相模原市,201,神奈川県,相模原市中央区,中央２丁目１１−１５
3,小田原市,201,神奈川県,小田原市,荻窪３００
4,茅ヶ崎市,201,神奈川県,茅ヶ崎市,茅ヶ崎１丁目１−１
5,逗子市,201,神奈川県,逗子市,逗子５丁目２−１６
6,南足柄市外二ケ市町組合,201,神奈川県,南足柄市,関本４４０
7,高座清掃施設組合,201,神奈川県,海老名市,本郷１−１
8,相模原市串川財産区,201,神奈川県,相模原市中央区,中央２丁目１１−１５
9,相模原市日連財産区,201,神奈川県,相模原市中央区,中央２丁目１１−１５


In [14]:
#Coping the dataset into another variable and randomly sample 20000 data inside the dataset
data_address_company = copy.deepcopy(data_address_company_original)
data_address_company = data_address_company.sample(20000).sort_index().reset_index(drop = True)
data_address_company = data_address_company.drop(["address_1"],axis = 1)

In [15]:
#making the columns which concats the address data
#makes the data into a latin alphabetical letter
#the latin alphabet is changed into the binary format
address_list = []
for i in range(len(data_address_company)):
    address_list.append(data_address_company.address_2[i] + data_address_company.address_3[i])
address_list = np.array(address_list)
data_address_company["address"] = address_list
address_english = []
for i in data_address_company.address:
    address_english.append(conv.do(mojimoji.zen_to_han(i))[:60])
address_english = np.array(address_english)
data_address_company["address_english"] = address_english
address_binary = []
for i in data_address_company["address_english"]:
    address_binary.append("".join(format(ord(j),"b").rjust(8,"0") for j in i).ljust(480,"0"))
address_binary = np.array(address_binary)
data_address_company["address_binary"] = address_binary
data_address_company = data_address_company.reset_index(drop = True)
data_address_company.head()

,company_name,company_group_id,address_2,address_3,address,address_english,address_binary
0,小田原市豊川財産区,201,小田原市,荻窪３００番地,小田原市荻窪３００番地,odawarashiogikubo300banchi,0110111101100100011000010111011101100001011100...
1,茅ケ崎シーサイド・パレス株式会社,301,藤沢市,辻堂２丁目４番２９−５０１号,藤沢市辻堂２丁目４番２９−５０１号,fujisawashitsujidou2choume4ban29−501gou,0110011001110101011010100110100101110011011000...
2,株式会社ＦＡＩＣ,301,横浜市青葉区,みすずが丘２番地１４,横浜市青葉区みすずが丘２番地１４,yokohamashiaobakumisuzugaoka2banchi14,0111100101101111011010110110111101101000011000...
3,アジアベスト株式会社,301,川崎市高津区,久本３丁目６番５−１５０１号,川崎市高津区久本３丁目６番５−１５０１号,kawasakishitakatsukuhisamoto3choume6ban5−1501gou,0110101101100001011101110110000101110011011000...
4,株式会社マテリアライズ,301,相模原市南区,上鶴間本町２丁目２４−５２,相模原市南区上鶴間本町２丁目２４−５２,sagamiharashiminamikukamitsurumahonchou2choume...,0111001101100001011001110110000101101101011010...


In [16]:
#making the backup of the dataset
data_address_company_backup = copy.deepcopy(data_address_company)

In [24]:
#dividing the binary into 48bits each and changing the 48bits into a float64 value
address_float_value = []
address_float = []
for j in range(0,len(data_address_company.address_binary[0]),48):
    tmp_float = []
    tmp_float_value = []
    for i,data in enumerate(data_address_company.address_binary):
        tmp_float.append(binary(np.float64(int(data[j:j+48],2)/(2**48-1)*1.7976931348623157e+308)))
        tmp_float_value.append(np.float64(int(data[j:j+48],2)/(2**48-1)*1.7976931348623157e+308))
    address_float.append(tmp_float)
    address_float_value.append(tmp_float_value)
for i in range(len(address_float)):
    data_address_company["address_float_{}".format(i)] = address_float[i]
    data_address_company["address_float_value_{}".format(i)] = address_float_value[i]
print(data_address_company.columns)
data_address_company = data_address_company.sort_values("address_float_value_1").reset_index(drop = True)
data_address_company.head()

Index(['company_name', 'company_group_id', 'address_2', 'address_3', 'address',
       'address_english', 'address_binary', 'address_float_0',
       'address_float_value_0', 'address_float_1', 'address_float_value_1',
       'address_float_2', 'address_float_value_2', 'address_float_3',
       'address_float_value_3', 'address_float_4', 'address_float_value_4',
       'address_float_5', 'address_float_value_5', 'address_float_6',
       'address_float_value_6', 'address_float_7', 'address_float_value_7',
       'address_float_8', 'address_float_value_8', 'address_float_9',
       'address_float_value_9'],
      dtype='object')


,company_name,company_group_id,address_2,address_3,address,address_english,address_binary,address_float_0,address_float_value_0,address_float_1,...,address_float_5,address_float_value_5,address_float_6,address_float_value_6,address_float_7,address_float_value_7,address_float_8,address_float_value_8,address_float_9,address_float_value_9
0,特定医療法人清輝会,399,小田原市,田島１２５番地,小田原市田島１２５番地,odawaraichidashima125banchi,0110111101100100011000010111011101100001011100...,0111111111011011110110010001100001011101110110...,7.822220e+307,0111111111011000010110100101100011011010000110...,...,0000000000000000000000000000000000000000000000...,0.000000e+00,0000000000000000000000000000000000000000000000...,0.000000e+00,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0
1,優幸不動産株式会社,301,小田原市,田島１２０４番地,小田原市田島１２０４番地,odawaraichidashima1204banchi,0110111101100100011000010111011101100001011100...,0111111111011011110110010001100001011101110110...,7.822220e+307,0111111111011000010110100101100011011010000110...,...,0000000000000000000000000000000000000000000000...,0.000000e+00,0000000000000000000000000000000000000000000000...,0.000000e+00,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0
2,有限会社アミーゴ,302,小田原市,田島９７８番地−９,小田原市田島９７８番地−９,odawaraichidashima978banchi−9,0110111101100100011000010111011101100001011100...,0111111111011011110110010001100001011101110110...,7.822220e+307,0111111111011000010110100101100011011010000110...,...,0000000000000000000000000000000000000000000000...,0.000000e+00,0000000000000000000000000000000000000000000000...,0.000000e+00,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0
3,株式会社駿河さがみ建装,301,小田原市,田島１５０４番地２,小田原市田島１５０４番地２,odawaraichidashima1504banchi2,0110111101100100011000010111011101100001011100...,0111111111011011110110010001100001011101110110...,7.822220e+307,0111111111011000010110100101100011011010000110...,...,0000000000000000000000000000000000000000000000...,0.000000e+00,0000000000000000000000000000000000000000000000...,0.000000e+00,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0
4,有限会社湘南企画,302,茅ヶ崎市,松浪１丁目５番１２号,茅ヶ崎市松浪１丁目５番１２号,chigasakiichimatsunami1choume5ban12gou,0110001101101000011010010110011101100001011100...,0111111111011000110110100001101001011001110110...,6.980657e+307,0111111111011000010110101101101001011010010110...,...,0111111111011000100110000101101110001100010011...,6.908520e+307,0111111111011011110111010100000000000000000000...,7.826779e+307,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0


## Anonymizing the dataset (k=20)

In [604]:
def make_not_anonymized_list_initial(data_dict,data,k):
    start = time.time()
    tmp = data["address_float_0"].unique()
    not_anonymized_list = []
    group_list = []
    for x,tm in enumerate(tmp):
        a = 0
        group = []
        for i in data_dict:
            if tm == data_dict[i]["address_float_0"]:
                data_dict[i]["group"] = [x]
                group.append(i)
                a += 1
        group_list.append(group)
        if a<k:
            not_anonymized_list.apeend(group)
    print(time.time()-start)
    print(len(not_anonymized_list))
    return data_dict

In [631]:
def make_group_list(data_dict,num):
    start = time.time()
    unique_float_list = []
    for i in data_dict:
        unique_float_list.append(data_dict[i]["address_float_{}".format(num)])
    unique_float_list = np.unique(np.array(unique_float_list))
    group_list_tmp = [data_dict[i]["group"] for i in range(len(data_dict))]
    #print(group_list_tmp)
    address_float_list = [data_dict[i]["address_float_{}".format(num)] for i in range(len(data_dict))]
    for x,unique in enumerate(unique_float_list):
        for i in data_dict:
            if unique == address_float_list[i]:
                group_list_tmp[i].append(x)
    print(time.time()-start)
    return group_list_tmp

In [606]:
def make_not_anonymized_and_group_list(data_dict,group_list_tmp,k):
    start = time.time()
    unique_group_list = []
    for i in data_dict:
        if data_dict[i]["group"] not in unique_group_list:
            unique_group_list.append(copy.deepcopy(data_dict[i]["group"]))
    group_list = []
    not_anonymized_list = []
    for unique in unique_group_list:
        group = []
        for x,tmp in enumerate(group_list_tmp):
            if tmp == unique:
                group.append(x)
        group_list.append(group)
        if 0 < len(group) < k:
            not_anonymized_list.append(group)
    print(time.time()-start)
    return [group_list, not_anonymized_list]

In [733]:
def make_anonymized_group_list(group_list,not_anonymized_list,data_dict,k,num):
    start = time.time()
    while len(not_anonymized_list) != 0:
        for not_anonymized in not_anonymized_list:
            diff = 0
            for x,i in enumerate(group_list):
                if data_dict[not_anonymized[0]]["group"] != data_dict[i[0]]["group"] and data_dict[not_anonymized[0]]["group"][:-1] == data_dict[i[0]]["group"][:-1]:
                    if diff == 0:
                        diff = abs(data_dict[not_anonymized[0]]["address_float_value_{}".format(num)]-data_dict[i[0]]["address_float_value_{}".format(num)])
                        closest_group = data_dict[i[0]]["group"][-1]
                        closest_group_num = x
                    elif diff > abs(data_dict[not_anonymized[0]]["address_float_value_{}".format(num)]-data_dict[i[0]]["address_float_value_{}".format(num)]):
                        diff = abs(data_dict[not_anonymized[0]]["address_float_value_{}".format(num)]-data_dict[i[0]]["address_float_value_{}".format(num)])
                        closest_group = data_dict[i[0]]["group"][-1]
                        closest_group_num = x
            #print(group_list[closest_group_num])
            for j in not_anonymized:
                #print(data_dict[j]["group"])
                group_list[closest_group_num].append(j)
                data_dict[j]["group"][-1] = closest_group
                #print(data_dict[j]["group"])
            group_list.remove(not_anonymized)
            break
        not_anonymized_list = []
        for group in group_list:
            #if 18292 in group:
                #print(group)
            if 0 < len(group) < k:
                not_anonymized_list.append(group)
                #print(anonymized_list)
        #print(time.time()-start)
        #print(len(group_list))
        #print(len(not_anonymized_list))
    print(time.time()-start)
    return group_list,data_dict

In [627]:
def make_anonymized_data_dict(group_list,data_dict,precision_num,num_1):
    for group in group_list:
        anonymized_data_list = []
        for x,i in enumerate(group):
            tmp = data_dict[i]["address_float_{}".format(num_1)]
            if x == 0:
                fix = data_dict[i]["address_float_{}".format(num_1)]
            elif fix != tmp:
                for k in range(1,precision_num):
                    if fix[0:-k] == tmp[0:-k]:
                        anonymized_data_list.append(fix[0:-k] + "".join("0" for l in range(k)))
                        break
        if len(anonymized_data_list)>0:
            for x,anony in enumerate(anonymized_data_list):
                if x == 0:
                    anonymized_data = anony
                elif anonymized_data != anony:
                    for k in range(1,precision_num):
                        if anonymized_data[0:-k] == anony[0:-k]:
                            anonymized_data = (anonymized_data[0:-k] + "".join("0" for l in range(k)))
                            break
            key_list = list(data_dict[0].keys())
            key_num = key_list.index("address_float_{}".format(num_1))
            for x,i in enumerate(group):
                data_dict[i]["address_float_{}".format(num_1)] = anonymized_data
                for num in range(key_num+1,len(key_list)-1):
                    if num%2 == 1:
                        data_dict[i][key_list[num]] = "".join("0" for l in range(precision_num))
                    else:
                        data_dict[i][key_list[num]] = np.float64(0)
    return data_dict

In [613]:
data_address_company_dict_backup = copy.deepcopy(data_address_company.drop(data_address_company.columns[0:4],axis = 1).T.to_dict())
data_address_company_dict = copy.deepcopy(data_address_company_dict_backup)

In [729]:
data_address_company_dict_for_use = copy.deepcopy(data_address_company_dict_backup)

In [730]:
start = time.time()
data_address_company_dict_1 = make_not_anonymized_list_initial(data_address_company_dict_for_use,data_address_company,20)
group_list_tmp = make_group_list(data_address_company_dict_1,1)
not_anonymized_and_group_list = make_not_anonymized_and_group_list(data_address_company_dict_1,group_list_tmp,20)
anonymized_group_list,data_address_company_anonymized_dict_1 = make_anonymized_group_list(not_anonymized_and_group_list[0],not_anonymized_and_group_list[1],data_address_company_dict_1,20,1)
data_address_company_anonymized_dict_1 = make_anonymized_data_dict(anonymized_group_list,data_address_company_anonymized_dict_1,64,1)
print(time.time()-start)

0.37870144844055176
0
0.48872971534729004
0.6431622505187988
0.04581952095031738
1.7762303352355957


In [734]:
for i in range(2,10):
    start = time.time()
    if i == 2:
        data_address_company_dict_start = copy.deepcopy(data_address_company_anonymized_dict_1)
    else:
        data_address_company_dict_start = copy.deepcopy(data_address_company_anonymized_dict_final)
    group_list_tmp = make_group_list(data_address_company_dict_start,i)
    not_anonymized_and_group_list = make_not_anonymized_and_group_list(data_address_company_dict_start,group_list_tmp,20)
    anonymized_group_list,data_address_company_anonymized_dict_final = make_anonymized_group_list(not_anonymized_and_group_list[0],not_anonymized_and_group_list[1],data_address_company_dict_start,20,i)
    data_address_company_anonymized_dict_final = make_anonymized_data_dict(anonymized_group_list,data_address_company_anonymized_dict_final,64,i)
    print(time.time()-start)

1.0853073596954346
1.6837561130523682
0.3017871379852295
5.079545259475708
1.761645793914795
2.4275481700897217
0.5186247825622559
6.671984672546387
1.1487298011779785
2.2287075519561768
0.3641090393066406
5.574630975723267
0.6319620609283447
1.5510680675506592
0.16548967361450195
4.396068572998047
0.34639501571655273
1.3502590656280518
0.08752870559692383
3.556009292602539
0.27111268043518066
1.2947800159454346
0.06723856925964355
3.436002492904663
0.07914900779724121
1.34201979637146
1.1920928955078125e-06
3.5338480472564697
0.0824899673461914
1.4161531925201416
1.430511474609375e-06
3.695039749145508


## validation

In [883]:
hoge = [data_address_company_anonymized_dict_final[i]["address_float_6"] for i in range(len(data_address_company_anonymized_dict_final))]

In [884]:
Counter(hoge)

Counter({'0000000000000000000000000000000000000000000000000000000000000000': 19585,
         '0111111111011010000110111101110101001100000000000000000000000000': 45,
         '0111111111011000110110100001101111011101010110110101100101011000': 130,
         '0111111111011011110111010100110000000000000000000000000000000000': 40,
         '0111111111011000110110100001101001001100000000000000000000000000': 60,
         '0111111111011001110110000101110011011010000110100100110000000000': 46,
         '0111111111011000010110110101100001011000110110100001101001010111': 22,
         '0111111111001000000000000000000000000000000000000000000000000000': 21,
         '0111111111011000010011000000000000000000000000000000000000000000': 22,
         '0111111111011010000110100100110000000000000000000000000000000000': 29})

In [885]:
data_address_company_anonymized_df_7.address_float_6.value_counts()

0000000000000000000000000000000000000000000000000000000000000000    19585
0111111111011000110110100001101111011101010110110101100101011000      130
0111111111011000110110100001101001001100000000000000000000000000       60
0111111111011001110110000101110011011010000110100100110000000000       46
0111111111011010000110111101110101001100000000000000000000000000       45
0111111111011011110111010100110000000000000000000000000000000000       40
0111111111011010000110100100110000000000000000000000000000000000       29
0111111111011000010011000000000000000000000000000000000000000000       22
0111111111011000010110110101100001011000110110100001101001010111       22
0111111111001000000000000000000000000000000000000000000000000000       21
Name: address_float_6, dtype: int64

## before the funtion

In [583]:
data_address_company_dict_backup = copy.deepcopy(data_address_company.drop(data_address_company.columns[0:4],axis = 1).T.to_dict())

In [595]:
data_address_company_dict = copy.deepcopy(data_address_company_dict_backup)

In [596]:
start = time.time()
tmp = data_address_company["address_float_0"].unique()
not_anonymized_list = []
group_list = []
for x,tm in enumerate(tmp):
    a = 0
    group = []
    for i in data_address_company_dict:
        if tm == data_address_company_dict[i]["address_float_0"]:
            data_address_company_dict[i]["group"] = [x]
            group.append(i)
            a += 1
    group_list.append(group)
    if a<20:
        not_anonymized_list.apeend(group)
print(time.time()-start)
print(len(not_anonymized_list))

0.42429566383361816
0


In [597]:
start = time.time()
unique_float_list = []
for i in data_address_company_dict:
    unique_float_list.append(data_address_company_dict[i]["address_float_1"])
unique_float_list = np.unique(np.array(unique_float_list))
group_list_tmp = [data_address_company_dict[i]["group"] for i in range(len(data_address_company_dict))]
address_float_list = [data_address_company_dict[i]["address_float_1"] for i in range(len(data_address_company_dict))]
for x,unique in enumerate(unique_float_list):
    for i in data_address_company_dict:
        if unique == address_float_list[i]:
            group_list_tmp[i].append(x)
print(time.time()-start)

1.1030609607696533


In [598]:
start = time.time()
unique_group_list = []
for i in data_address_company_dict:
    if data_address_company_dict[i]["group"] not in unique_group_list:
        unique_group_list.append(copy.deepcopy(data_address_company_dict[i]["group"]))
group_list = []
not_anonymized_list = []
#not_anonymized_unique_group_list = []
for unique in unique_group_list:
    group = []
    for x,tmp in enumerate(group_list_tmp):
        if tmp == unique:
            group.append(x)
    group_list.append(group)
    if 0 < len(group) < 20:
        not_anonymized_list.append(group)
        #not_anonymized_unique_group_list.append(unique)
print(time.time()-start)

1.2354121208190918


In [599]:
while len(not_anonymized_list) != 0:
    start = time.time()
    for not_anonymized in not_anonymized_list:
        diff = 0
        for x,i in enumerate(group_list):
            if data_address_company_dict[not_anonymized[0]]["group"] != data_address_company_dict[i[0]]["group"] and data_address_company_dict[not_anonymized[0]]["group"][:-1] == data_address_company_dict[i[0]]["group"][:-1]:
                if diff == 0:
                    diff = abs(data_address_company_dict[not_anonymized[0]]["address_float_value_1"]-data_address_company_dict[i[0]]["address_float_value_1"])
                    closest_group = data_address_company_dict[i[0]]["group"][-1]
                    closest_group_num = x
                elif diff > abs(data_address_company_dict[not_anonymized[0]]["address_float_value_1"]-data_address_company_dict[i[0]]["address_float_value_1"]):
                    diff = abs(data_address_company_dict[not_anonymized[0]]["address_float_value_1"]-data_address_company_dict[i[0]]["address_float_value_1"])
                    closest_group = data_address_company_dict[i[0]]["group"][-1]
                    closest_group_num = x
        #print(group_list[closest_group_num])
        for j in not_anonymized:
            #print(data_address_company_dict[j]["group"])
            group_list[closest_group_num].append(j)
            data_address_company_dict[j]["group"][-1] = closest_group
            #print(data_address_company_dict[j]["group"])
        group_list.remove(not_anonymized)
        break
    """
    unique_group_list = []
    for i in data_address_company_dict:
        if data_address_company_dict[i[0]]["group"] not in unique_group_list:
            unique_group_list.append(data_address_company_dict[i[0]]["group"])
    """
    not_anonymized_list = []
    for group in group_list:
        if 0 < len(group) < 20:
            not_anonymized_list.append(group)
            #print(anonymized_list)
    print(time.time()-start)
    #print(len(group_list))
    #print(len(not_anonymized_list))

0.0009200572967529297
0.0008466243743896484
0.000843048095703125
0.0010187625885009766
0.0005710124969482422
0.0005803108215332031
0.0005552768707275391
0.0005521774291992188
0.0005486011505126953
0.0005509853363037109
0.0005404949188232422
0.0005404949188232422
0.0007810592651367188
0.00043320655822753906
0.00043511390686035156
0.0004315376281738281
0.0004107952117919922
0.00042128562927246094
0.0004115104675292969
0.00042629241943359375
0.0004069805145263672
0.0004172325134277344
0.00042724609375
0.00040411949157714844
0.00041604042053222656
0.0004146099090576172
0.00047588348388671875
0.0005280971527099609
0.0005171298980712891
0.0004932880401611328
0.0004851818084716797
0.0004901885986328125
0.0004515647888183594
0.0004417896270751953
0.00044226646423339844
0.00041294097900390625
0.0004439353942871094
0.00043392181396484375
0.0004398822784423828
0.00042366981506347656
0.00041675567626953125
0.00042247772216796875
0.0005652904510498047
0.0003809928894042969
0.0003821849822998047
0.0

In [600]:
for group in group_list:
    anonymized_data_list = []
    for x,i in enumerate(group):
        tmp = data_address_company_dict[i]["address_float_1"]
        if x == 0:
            fix = data_address_company_dict[i]["address_float_1"]
        elif fix != tmp:
            for k in range(1,64):
                if fix[0:-k] == tmp[0:-k]:
                    anonymized_data_list.append(fix[0:-k] + "".join("0" for l in range(k)))
                    break
    if len(anonymized_data_list)>0:
        for x,anony in enumerate(anonymized_data_list):
            if x == 0:
                anonymized_data = anony
            elif anonymized_data != anony:
                for k in range(1,64):
                    if anonymized_data[0:-k] == anony[0:-k]:
                        anonymized_data = (anonymized_data[0:-k] + "".join("0" for l in range(k)))
                        break
        key_list = list(data_address_company_dict[0].keys())
        key_num = key_list.index("address_float_1")
        for x,i in enumerate(group):
            data_address_company_dict[i]["address_float_1"] = anonymized_data
            for num in range(key_num+1,len(key_list)):
                if num%2 == 1:
                    data_address_company_dict[i][key_list[num]] = "".join("0" for l in range(64))
                else:
                    data_address_company_dict[i][key_list[num]] = np.float64(0)

## list version

In [827]:
#look at the first column and see if there is any group which has a data less than k
group_0 = []
tmp = data_address_company["address_float_0"].unique()
for j in tmp:
    gro_0 = []
    for i,data in enumerate(data_address_company.address_float_0):
        if data == j:
            gro_0.append(i)
    group_0.append(gro_0)
for i in group_0:
    if 0<len(i)<20:
        print(len(i))
        print(i)

In [828]:
group_1 = []
tmp = data_address_company["address_float_1"].unique()
tmp_1 = data_address_company.address_float_1.values
for k in group_0:
    grou_1 = []
    for j in tmp:
        gro_1 = []
        for m in k:
            if tmp_1[m] == j:
                gro_1.append(m)
        if len(gro_1) != 0:
            grou_1.append(gro_1)
    group_1.append(grou_1)

In [829]:
group_1_backup = copy.deepcopy(group_1)

In [830]:
group_1_not_anonymized = []
for x,i in enumerate(group_1):
        for j in i:
            if 0<len(j)<20:
                #print(len(j))
                #print(j)
                group_1_not_anonymized.append([x,j])
                #if j not in not_anonymized_list:
                    #print(nooooooo)

In [831]:
tmp = data_address_company.address_float_value_1
while len(group_1_not_anonymized) != 0:
    a = 0
    start = time.time()
#for i in group_1_not_anonymized:
    for x,j in enumerate(group_1_backup[group_1_not_anonymized[0][0]]):
        if len(j) != 0:
            if j[0] != group_1_not_anonymized[0][1][0]:
                if a == 0:
                    diff = abs(tmp[j[0]]-tmp[group_1_not_anonymized[0][1][0]])
                    closest_group = x
                    a += 1
                elif diff > abs(tmp[j[0]]-tmp[group_1_not_anonymized[0][1][0]]):
                    #print(diff)
                    #print(tmp[j[0]]-tmp[group_1_not_anonymized[0][1][0]])
                    diff = abs(tmp[j[0]]-tmp[group_1_not_anonymized[0][1][0]])
                    closest_group = x
    #print(group_1_backup[group_1_not_anonymized[0][0]][closest_group])
    group_1_backup[group_1_not_anonymized[0][0]][closest_group].extend(group_1_not_anonymized[0][1])
    group_1_backup[group_1_not_anonymized[0][0]].remove(group_1_not_anonymized[0][1])
    group_1_not_anonymized = []
    for x,i in enumerate(group_1_backup):
        for j in i:
            if 0<len(j)<20:
                #print(len(j))
                #print(j)
                group_1_not_anonymized.append([x,j])
    #if len(group_1_not_anonymized) != 0:
        #print(len(group_1_not_anonymized))
    #print(time.time()-start)

In [832]:
start = time.time()
data_address_company_anonymized = copy.deepcopy(data_address_company.values)
column = list(data_address_company.columns)
tmp = data_address_company["address_float_1"].values
for y in group_1_backup:
    start_3 = time.time()
    for i in y:
        a = 0
        start_1 = time.time()
        anonymized_data = []
        for x,j in enumerate(i):
            if x == 0:
                fix = tmp[j]
            elif fix != tmp[j]:
                a+=1
                for k in range(1,64):
                    if fix[0:-k] == tmp[j][0:-k]:
                        anonymized_data.append(fix[0:-k] + "".join("0" for l in range(k)))
                        break
        if a > 0:
            for x,anony in enumerate(anonymized_data):
                if x == 0:
                    fix = anony
                elif fix != anony:
                    for k in range(1,64):
                        if fix[0:-k] == anony[0:-k]:
                            fix = (fix[0:-k] + "".join("0" for l in range(k)))
                            break
            #print(i)
            #print(fix)
            #print("yeah")
            column_num = column.index("address_float_1")
            for m in i:
                data_address_company_anonymized[m,column_num] = fix
                for o in range(column_num+1,len(column)):
                    if o%2 == 1:
                        data_address_company_anonymized[m,o] = "".join("0" for l in range(64))
                    else:
                        data_address_company_anonymized[m,o] = np.float64(0)
        #print("loop_1=",time.time()-start_1)
    #print("loop_3=",time.time()-start_3)
#print(time.time()-start)

In [833]:
data_address_company_anonymized_df = pd.DataFrame(data_address_company_anonymized,columns=column)

In [834]:
group_2 = []
tmp = data_address_company_anonymized_df["address_float_2"].unique()
tmp_1 = data_address_company_anonymized_df.address_float_2.values
for i in group_1_backup:
    grou_2 = []
    for x,j in enumerate(i):
        gro_2 = []
        for l in tmp:
            gr_2 = []
            for k in j:
                if tmp_1[k] == l:
                    gr_2.append(k)
            if len(gr_2) != 0:
                gro_2.append(gr_2)
        #print(gro_2)
        grou_2.append(gro_2)
    group_2.append(grou_2)

In [835]:
group_2_not_anonymized = []
for x,i in enumerate(group_2):
        for y,j in enumerate(i):
            for k in j:
                if 0<len(k)<20:
                    #print(x,y)
                    #print(len(k))
                    #print(k)
                    group_2_not_anonymized.append([x,y,k])

In [836]:
group_2_backup = copy.deepcopy(group_2)

In [837]:
tmp = data_address_company_anonymized_df.address_float_value_2
while len(group_2_not_anonymized) != 0:
    a = 0
    start = time.time()
#for i in group_2_not_anonymized:
    for x,j in enumerate(group_2_backup[group_2_not_anonymized[0][0]][group_2_not_anonymized[0][1]]):
        if len(j) != 0:
            if j[0] != group_2_not_anonymized[0][2][0]:
                if a == 0:
                    diff = abs(tmp[j[0]]-tmp[group_2_not_anonymized[0][2][0]])
                    closest_group = x
                    a += 1
                elif diff > abs(tmp[j[0]]-tmp[group_2_not_anonymized[0][2][0]]):
                    #print(diff)
                    #print(tmp[j[0]]-tmp[group_2_not_anonymized[0][1][0]])
                    diff = abs(tmp[j[0]]-tmp[group_2_not_anonymized[0][2][0]])
                    closest_group = x
    group_2_backup[group_2_not_anonymized[0][0]][group_2_not_anonymized[0][1]][closest_group].extend(group_2_not_anonymized[0][2])
    group_2_backup[group_2_not_anonymized[0][0]][group_2_not_anonymized[0][1]].remove(group_2_not_anonymized[0][2])
    group_2_not_anonymized = []
    for x,i in enumerate(group_2_backup):
            for y,j in enumerate(i):
                for k in j:
                    if 18478 in k:
                        print(k)
                    if 0<len(k)<20:
                        #print(x,y)
                        #print(len(k))
                        #print(k)
                        group_2_not_anonymized.append([x,y,k])
    #if len(group_2_not_anonymized) != 0:
        #print(group_2_not_anonymized[0])
    #print(time.time()-start)

[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734,

[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734, 18811]
[18478, 18701, 18706, 18734,

In [838]:
start = time.time()
data_address_company_anonymized_1 = copy.deepcopy(data_address_company_anonymized_df.values)
column = list(data_address_company_anonymized_df.columns)
tmp = data_address_company_anonymized_df["address_float_2"].values
for q in group_2_backup:
    start_4 = time.time()
    for y in q:
        start_3 = time.time()
        for i in y:
            a = 0
            start_1 = time.time()
            anonymized_data = []
            for x,j in enumerate(i):
                if x == 0:
                    fix = tmp[j]
                elif fix != tmp[j]:
                    a+=1
                    for k in range(1,64):
                        if fix[0:-k] == tmp[j][0:-k]:
                            anonymized_data.append(fix[0:-k] + "".join("0" for l in range(k)))
                            break
            if a > 0:
                for x,anony in enumerate(anonymized_data):
                    if x == 0:
                        fix = anony
                    elif fix != anony:
                        for k in range(1,64):
                            if fix[0:-k] == anony[0:-k]:
                                fix = (fix[0:-k] + "".join("0" for l in range(k)))
                                break
                print("yeah")
                column_num = column.index("address_float_2")
                for m in i:
                    data_address_company_anonymized_1[m,column_num] = fix
                    for o in range(column_num+1,len(column)):
                        if o%2 == 1:
                            data_address_company_anonymized_1[m,o] = "".join("0" for l in range(64))
                        else:
                            data_address_company_anonymized_1[m,o] = np.float64(0)
                print("loop_2=",time.time()-start_1)            
            print("loop_1=",time.time()-start_1)
        print("loop_3=",time.time()-start_3)
    print("loop_4=",time.time()-start_4)
print(time.time()-start)

loop_1= 3.62396240234375e-05
loop_3= 0.00018358230590820312
loop_1= 2.7179718017578125e-05
loop_3= 0.0001595020294189453
yeah
loop_2= 0.004283428192138672
loop_1= 0.004645109176635742
loop_3= 0.0049016475677490234
yeah
loop_2= 0.004145383834838867
loop_1= 0.004255771636962891
loop_3= 0.0043256282806396484
loop_1= 2.2649765014648438e-05
loop_3= 9.989738464355469e-05
yeah
loop_2= 0.0027964115142822266
loop_1= 0.0029861927032470703
loop_3= 0.0030510425567626953
loop_1= 2.384185791015625e-05
loop_3= 8.940696716308594e-05
loop_1= 2.002716064453125e-05
loop_3= 8.273124694824219e-05
loop_1= 1.3828277587890625e-05
loop_3= 7.62939453125e-05
loop_1= 1.3828277587890625e-05
loop_3= 7.62939453125e-05
yeah
loop_2= 0.003981113433837891
loop_1= 0.004041194915771484
loop_3= 0.004216432571411133
loop_1= 2.2172927856445312e-05
loop_3= 8.654594421386719e-05
loop_4= 0.01886272430419922
loop_1= 1.1444091796875e-05
loop_3= 7.319450378417969e-05
yeah
loop_2= 0.00455784797668457
loop_1= 0.0046999454498291016
y

loop_3= 0.0023632049560546875
yeah
loop_2= 0.002534627914428711
loop_1= 0.0027039051055908203
yeah
loop_2= 0.002293109893798828
loop_1= 0.002382993698120117
loop_3= 0.005223751068115234
yeah
loop_2= 0.005654573440551758
loop_1= 0.005789518356323242
loop_3= 0.005915641784667969
yeah
loop_2= 0.004809379577636719
loop_1= 0.0048825740814208984
yeah
loop_2= 0.0018384456634521484
loop_1= 0.001905202865600586
loop_3= 0.006913661956787109
loop_1= 3.337860107421875e-05
loop_3= 0.0001125335693359375
loop_1= 1.9788742065429688e-05
loop_1= 1.4066696166992188e-05
yeah
loop_2= 0.011301994323730469
loop_1= 0.011397600173950195
yeah
loop_2= 0.0027718544006347656
loop_1= 0.003183603286743164
yeah
loop_2= 0.002882242202758789
loop_1= 0.0029730796813964844
loop_3= 0.01870441436767578
loop_1= 2.47955322265625e-05
loop_3= 0.00010395050048828125
yeah
loop_2= 0.0030655860900878906
loop_1= 0.0031812191009521484
loop_3= 0.003256559371948242
yeah
loop_2= 0.006730556488037109
loop_1= 0.0068471431732177734
loop_3

In [839]:
data_address_company_anonymized_df_1 = pd.DataFrame(data_address_company_anonymized_1,columns=column)

In [840]:
group_3 = []
tmp = list(data_address_company_anonymized_df_1["address_float_3"].unique())
tmp_1 = list(data_address_company_anonymized_df_1.address_float_3.values)
for m in group_2_backup:
    start = time.time()
    grou_3 = []
    for i in m:
        gro_3 = []
        for x,j in enumerate(i):
            gr_3 = []
            for l in tmp:
                g_3 = []
                for k in j:
                    if tmp_1[k] == l:
                        g_3.append(k)
                if len(g_3) != 0:
                    gr_3.append(g_3)
            #print(gro_3)
            gro_3.append(gr_3)
        grou_3.append(gro_3)
    group_3.append(grou_3)
    print(time.time()-start)

0.05756402015686035
0.05656027793884277
0.02517986297607422
0.01327204704284668
0.17841339111328125
0.02442789077758789
0.02837204933166504
0.012354612350463867
0.01850605010986328
0.026359081268310547
0.10316014289855957
0.40284156799316406
1.2286779880523682
0.014797449111938477
0.03760552406311035
0.06269288063049316
0.07585954666137695
0.03872847557067871
0.07015371322631836
0.07458877563476562
0.016059160232543945
0.015736103057861328
0.014567136764526367
0.1368725299835205


In [841]:
group_3_backup = copy.deepcopy(group_3)

In [842]:
group_3_not_anonymized = []
for x,m in enumerate(group_3):
    for y,i in enumerate(m):
            for z,j in enumerate(i):
                for k in j:
                    if 0<len(k)<20:
                        print(x,y,z)
                        print(len(k))
                        print(k)
                        group_3_not_anonymized.append([x,y,z,k])

1 0 0
1
[4]
1 0 0
1
[5]
1 0 0
2
[6, 15]
1 0 0
4
[7, 9, 10, 21]
1 0 0
2
[8, 19]
1 0 0
8
[11, 12, 13, 16, 17, 20, 22, 23]
1 0 0
2
[14, 18]
1 0 0
1
[24]
4 0 0
10
[1357, 1446, 1514, 1575, 1677, 2227, 2247, 2272, 2308, 2370]
4 0 0
18
[1104, 1113, 1133, 1361, 1421, 1578, 1626, 1653, 1690, 1718, 1729, 1800, 1858, 1915, 2055, 2220, 2229, 2429]
4 0 0
7
[1236, 1237, 1276, 1960, 2053, 2368, 2412]
4 0 0
5
[1115, 1314, 1548, 2267, 2386]
4 0 0
8
[1139, 1264, 2071, 2191, 2296, 2356, 2430, 2443]
4 0 0
1
[1153]
4 0 0
7
[1199, 1398, 1558, 1934, 1966, 2298, 2417]
4 0 0
8
[1202, 1233, 1532, 1965, 2036, 2206, 2381, 2442]
4 0 0
6
[1206, 1702, 1863, 1988, 2171, 2211]
4 0 0
6
[1210, 1213, 1451, 1912, 2056, 2314]
4 0 0
9
[1215, 1230, 1281, 1292, 1545, 1921, 1986, 2352, 2395]
4 0 0
17
[1234, 1235, 1536, 1537, 1559, 1600, 1623, 1839, 1964, 2027, 2074, 2284, 2317, 2383, 2393, 2439, 2441]
4 0 0
5
[1238, 1379, 1397, 1954, 2011]
4 0 0
4
[1239, 1533, 2173, 2228]
4 0 0
4
[1240, 1531, 1956, 2130]
4 0 0
4
[1241, 1593, 1

11 4 0
17
[5916, 5934, 5953, 5960, 5992, 6019, 6020, 6036, 6058, 6062, 6072, 6080, 6086, 6095, 6168, 6170, 6219]
11 4 0
13
[5920, 5921, 5946, 5969, 6022, 6037, 6079, 6098, 6130, 6132, 6165, 6186, 6191]
11 4 0
7
[5922, 5945, 5996, 6010, 6041, 6143, 6177]
11 4 0
5
[5923, 6032, 6050, 6092, 6161]
11 4 0
5
[5924, 5947, 6011, 6027, 6035]
11 4 0
3
[5925, 6007, 6049]
11 4 0
18
[5926, 5935, 5939, 5955, 5973, 6013, 6068, 6075, 6077, 6100, 6109, 6129, 6153, 6159, 6166, 6172, 6189, 6201]
11 4 0
15
[5927, 5932, 5937, 5974, 5986, 5989, 5991, 6009, 6015, 6076, 6084, 6139, 6211, 6216, 6222]
11 4 0
12
[5930, 5933, 5961, 5972, 5987, 5988, 6023, 6024, 6048, 6097, 6175, 6187]
11 4 0
3
[5938, 6103, 6185]
11 4 0
12
[5940, 5951, 5976, 5993, 6001, 6006, 6053, 6102, 6163, 6206, 6207, 6208]
11 4 0
18
[5941, 5943, 5950, 5965, 5967, 6000, 6004, 6030, 6063, 6071, 6099, 6110, 6122, 6145, 6176, 6178, 6179, 6196]
11 4 0
1
[5963]
11 4 0
2
[5964, 6066]
11 4 0
6
[5971, 5990, 6028, 6043, 6089, 6199]
11 4 0
3
[5997, 6085,

[9128, 9211, 9217, 9224, 9299, 9359, 9390, 9552, 9657, 9694, 9864, 10004, 10036, 10257, 10353, 10442]
12 3 1
18
[9134, 9226, 9516, 9545, 9556, 9565, 9656, 9707, 9861, 9873, 9874, 9883, 9915, 9943, 10079, 10115, 10120, 10450]
12 3 1
9
[9182, 9541, 9938, 9972, 10000, 10299, 10347, 10397, 10480]
12 3 1
6
[9213, 9355, 9621, 9632, 9885, 10112]
12 3 1
12
[9223, 9304, 9334, 9633, 9695, 9702, 9911, 10068, 10219, 10363, 10420, 10518]
12 3 1
4
[9328, 9533, 9661, 10039]
12 3 1
18
[9345, 9476, 9532, 9579, 9717, 9794, 9866, 9882, 9988, 10007, 10202, 10210, 10305, 10321, 10391, 10399, 10446, 10588]
12 3 1
9
[9353, 9572, 9646, 9654, 9870, 9888, 10016, 10082, 10297]
12 3 1
3
[9408, 9858, 10317]
12 3 1
11
[9443, 9690, 9745, 9764, 9772, 9907, 9928, 9934, 10100, 10441, 10478]
12 3 1
4
[9513, 9604, 9734, 10337]
12 3 1
4
[9583, 9895, 10146, 10520]
12 3 1
3
[9668, 10483, 10532]
12 3 1
2
[9798, 10375]
12 3 1
3
[9821, 9828, 10298]
12 3 1
3
[10011, 10345, 10408]
12 3 1
2
[10035, 10456]
12 3 2
7
[9179, 9406, 97

[15246]
12 7 0
5
[15264, 15456, 15815, 15958, 16044]
12 7 0
1
[15370]
12 7 0
1
[15804]
12 7 0
5
[15830, 15855, 15930, 16011, 16091]
12 7 1
11
[15144, 15242, 15314, 15377, 15582, 15604, 15723, 15729, 15828, 15972, 16100]
12 7 1
12
[15272, 15365, 15546, 15586, 15725, 15809, 15810, 15849, 15885, 15897, 16034, 16132]
12 7 1
6
[15352, 15558, 15763, 15861, 15873, 16046]
12 7 1
13
[15084, 15286, 15326, 15329, 15337, 15410, 15424, 15851, 15859, 15916, 15992, 16107, 16122]
12 7 1
18
[15090, 15145, 15259, 15304, 15305, 15346, 15402, 15438, 15497, 15521, 15732, 15871, 15888, 15997, 16003, 16054, 16055, 16131]
12 7 1
16
[15099, 15155, 15158, 15211, 15257, 15291, 15390, 15465, 15542, 15697, 15701, 15760, 15839, 15862, 15970, 16124]
12 7 1
17
[15100, 15222, 15307, 15330, 15383, 15449, 15529, 15574, 15693, 15709, 15731, 15743, 15814, 15867, 15974, 16017, 16134]
12 7 1
6
[15101, 15446, 15495, 15737, 15940, 15948]
12 7 1
16
[15223, 15399, 15413, 15511, 15527, 15537, 15552, 15584, 15644, 15668, 15720, 1

In [843]:
tmp = data_address_company_anonymized_df_1.address_float_value_3
while len(group_3_not_anonymized) != 0:
    a = 0
    start = time.time()
#for i in group_3_not_anonymized:
    for x,j in enumerate(group_3_backup[group_3_not_anonymized[0][0]][group_3_not_anonymized[0][1]][group_3_not_anonymized[0][2]]):
        if len(j) != 0:
            if j[0] != group_3_not_anonymized[0][3][0]:
                if a == 0:
                    diff = abs(tmp[j[0]]-tmp[group_3_not_anonymized[0][3][0]])
                    closest_group = x
                    a += 1
                elif diff > abs(tmp[j[0]]-tmp[group_3_not_anonymized[0][3][0]]):
                    #print(diff)
                    #print(tmp[j[0]]-tmp[group_3_not_anonymized[0][1][0]])
                    diff = abs(tmp[j[0]]-tmp[group_3_not_anonymized[0][3][0]])
                    closest_group = x
    group_3_backup[group_3_not_anonymized[0][0]][group_3_not_anonymized[0][1]][group_3_not_anonymized[0][2]][closest_group].extend(group_3_not_anonymized[0][3])
    group_3_backup[group_3_not_anonymized[0][0]][group_3_not_anonymized[0][1]][group_3_not_anonymized[0][2]].remove(group_3_not_anonymized[0][3])
    group_3_not_anonymized = []
    for x,m in enumerate(group_3_backup):
        for y,i in enumerate(m):
                for z,j in enumerate(i):
                    for k in j:
                        if 0<len(k)<20:
                            #print(x,y,z)
                            #print(len(k))
                            #print(k)
                            group_3_not_anonymized.append([x,y,z,k])
    if len(group_3_not_anonymized) != 0:
        print(group_3_not_anonymized[0])
    print(time.time()-start)

[1, 0, 0, [5]]
0.0023453235626220703
[1, 0, 0, [6, 15]]
0.0015573501586914062
[1, 0, 0, [7, 9, 10, 21, 4, 6, 15]]
0.0011332035064697266
[1, 0, 0, [8, 19]]
0.001506805419921875
[1, 0, 0, [11, 12, 13, 16, 17, 20, 22, 23, 8, 19]]
0.0008091926574707031
[1, 0, 0, [14, 18, 11, 12, 13, 16, 17, 20, 22, 23, 8, 19]]
0.0009927749633789062
[4, 0, 0, [1357, 1446, 1514, 1575, 1677, 2227, 2247, 2272, 2308, 2370]]
0.0012238025665283203
[4, 0, 0, [1104, 1113, 1133, 1361, 1421, 1578, 1626, 1653, 1690, 1718, 1729, 1800, 1858, 1915, 2055, 2220, 2229, 2429]]
0.0037958621978759766
[4, 0, 0, [1236, 1237, 1276, 1960, 2053, 2368, 2412]]
0.0021827220916748047
[4, 0, 0, [1139, 1264, 2071, 2191, 2296, 2356, 2430, 2443]]
0.0025603771209716797
[4, 0, 0, [1153, 1357, 1446, 1514, 1575, 1677, 2227, 2247, 2272, 2308, 2370]]
0.0019998550415039062
[4, 0, 0, [1199, 1398, 1558, 1934, 1966, 2298, 2417]]
0.001718759536743164
[4, 0, 0, [1202, 1233, 1532, 1965, 2036, 2206, 2381, 2442]]
0.002735614776611328
[4, 0, 0, [1206, 170

[11, 4, 0, [5927, 5932, 5937, 5974, 5986, 5989, 5991, 6009, 6015, 6076, 6084, 6139, 6211, 6216, 6222]]
0.001310586929321289
[11, 4, 0, [5930, 5933, 5961, 5972, 5987, 5988, 6023, 6024, 6048, 6097, 6175, 6187]]
0.0016067028045654297
[11, 4, 0, [5938, 6103, 6185]]
0.001950979232788086
[11, 4, 0, [5963, 5925, 6007, 6049]]
0.0011599063873291016
[11, 4, 0, [5964, 6066, 5916, 5934, 5953, 5960, 5992, 6019, 6020, 6036, 6058, 6062, 6072, 6080, 6086, 6095, 6168, 6170, 6219]]
0.0012483596801757812
[11, 4, 0, [5971, 5990, 6028, 6043, 6089, 6199, 5922, 5945, 5996, 6010, 6041, 6143, 6177, 5912, 5968, 6118, 6136, 6141, 6190]]
0.001554250717163086
[11, 4, 0, [5997, 6085, 6113]]
0.0010218620300292969
[11, 4, 0, [6038, 6115, 5913, 6107, 6183, 5938, 6103, 6185]]
0.0009541511535644531
[11, 4, 0, [6054, 6152, 6171]]
0.0010292530059814453
[11, 4, 0, [6120, 6134, 6164, 6197, 6212, 6054, 6152, 6171]]
0.0010809898376464844
[11, 5, 0, [6290, 6359, 6384, 6386, 6393, 6397, 6421, 6510, 6610, 6681, 6764, 6769, 6829,

[12, 3, 0, [9429, 9682]]
0.006579875946044922
[12, 3, 0, [9131, 10099, 10510, 9123, 9258, 9292, 9389, 9409, 9447, 9451, 9504, 9555, 9591, 9640, 9792, 9908, 10174, 10390]]
0.0024640560150146484
[12, 3, 0, [9137, 9178, 9199, 9240, 10008, 10032, 10343, 10470, 10587]]
0.002259969711303711
[12, 3, 0, [9140, 9200, 9250, 9301, 9483, 9746, 10047, 10060, 10061, 10125, 10260, 10394, 10484, 10552, 10560, 10561]]
0.0023331642150878906
[12, 3, 0, [9142, 9323, 9737, 9838, 9900, 9967, 10081, 10123, 10207, 10312, 10380, 10393]]
0.0028421878814697266
[12, 3, 0, [9144, 9373, 9430, 9577]]
0.0021910667419433594
[12, 3, 0, [9145, 9340, 9489, 9608, 9744, 9766, 10243, 10536]]
0.0021660327911376953
[12, 3, 0, [9377, 10199]]
0.002215147018432617
[12, 3, 0, [9153, 9618, 10373, 10465, 9145, 9340, 9489, 9608, 9744, 9766, 10243, 10536]]
0.0021386146545410156
[12, 3, 0, [9183, 9445, 9601]]
0.0019712448120117188
[12, 3, 0, [9187, 9511, 9791, 9834, 9975, 10238, 10351, 9142, 9323, 9737, 9838, 9900, 9967, 10081, 10123,

[12, 4, 2, [11177, 11618, 12453, 12459, 12488, 12558, 12657, 10840, 10917, 10946, 11140, 11237, 12027, 12122, 12673]]
0.0016112327575683594
[12, 4, 2, [11776, 11155, 12013, 12234, 12357]]
0.0013082027435302734
[12, 4, 2, [12057, 12487, 12664, 10808, 11337, 11526, 11731, 12231, 12237, 12620, 10692, 10699, 12017, 12146]]
0.0014529228210449219
[12, 4, 2, [12252, 10726, 10910, 11931, 12059, 12057, 12487, 12664, 10808, 11337, 11526, 11731, 12231, 12237, 12620, 10692, 10699, 12017, 12146]]
0.0011949539184570312
[12, 4, 3, [10623, 10893, 10916, 10979, 11004, 11095, 11350, 11483, 11578, 11598, 11652, 11748, 11844, 11883, 12035, 12283, 12382, 12486, 12559]]
0.0015168190002441406
[12, 4, 3, [10608, 10700, 10716, 10744, 10865, 10888, 10915, 10930, 11234, 11338, 11481, 11707, 11894, 11956, 12055, 12080, 12354, 12424, 12621]]
0.0011224746704101562
[12, 4, 3, [10613, 11078, 11191, 11223, 11372, 11587, 11783, 11890]]
0.001192331314086914
[12, 4, 3, [10672, 10794, 10797]]
0.0012106895446777344
[12, 4,

[12, 7, 0, [15199, 15254, 15263, 15284, 15349, 15645, 15768, 15816, 15835, 15950]]
0.057272911071777344
[12, 7, 0, [15209, 15108, 15481, 15566]]
0.0019583702087402344
[12, 7, 0, [15246]]
0.002064943313598633
[12, 7, 0, [15264, 15456, 15815, 15958, 16044]]
0.0013284683227539062
[12, 7, 0, [15370, 15802]]
0.0012056827545166016
[12, 7, 0, [15804, 15121, 15196, 15443, 15576, 16019, 16042]]
0.0011441707611083984
[12, 7, 0, [15830, 15855, 15930, 16011, 16091, 15252, 15538, 15548, 15682, 15882, 16093, 15264, 15456, 15815, 15958, 16044]]
0.0016279220581054688
[12, 7, 1, [15144, 15242, 15314, 15377, 15582, 15604, 15723, 15729, 15828, 15972, 16100]]
0.0011692047119140625
[12, 7, 1, [15272, 15365, 15546, 15586, 15725, 15809, 15810, 15849, 15885, 15897, 16034, 16132]]
0.0016241073608398438
[12, 7, 1, [15352, 15558, 15763, 15861, 15873, 16046]]
0.0015978813171386719
[12, 7, 1, [15084, 15286, 15326, 15329, 15337, 15410, 15424, 15851, 15859, 15916, 15992, 16107, 16122]]
0.0015053749084472656
[12, 7, 

In [844]:
start = time.time()
data_address_company_anonymized_2 = copy.deepcopy(data_address_company_anonymized_df_1.values)
column = list(data_address_company_anonymized_df_1.columns)
tmp = data_address_company_anonymized_df_1["address_float_3"].values
for p in group_3_backup:
    start_5 = time.time()
    for q in p:
        start_4 = time.time()
        for y in q:
            start_3 = time.time()
            for i in y:
                a = 0
                start_1 = time.time()
                anonymized_data = []
                for x,j in enumerate(i):
                    if x == 0:
                        fix = tmp[j]
                    elif fix != tmp[j]:
                        a+=1
                        for k in range(1,64):
                            if fix[0:-k] == tmp[j][0:-k]:
                                anonymized_data.append(fix[0:-k] + "".join("0" for l in range(k)))
                                break
                if a > 0:
                    for x,anony in enumerate(anonymized_data):
                        if x == 0:
                            fix = anony
                        elif fix != anony:
                            for k in range(1,64):
                                if fix[0:-k] == anony[0:-k]:
                                    fix = (fix[0:-k] + "".join("0" for l in range(k)))
                                    break
                    print("yeah")
                    column_num = column.index("address_float_3")
                    for m in i:
                        data_address_company_anonymized_2[m,column_num] = fix
                        for o in range(column_num+1,len(column)):
                            if o%2 == 1:
                                data_address_company_anonymized_2[m,o] = "".join("0" for l in range(64))
                            else:
                                data_address_company_anonymized_2[m,o] = np.float64(0)
                    print("loop_2=",time.time()-start_1)            
                print("loop_1=",time.time()-start_1)
            print("loop_3=",time.time()-start_3)
        print("loop_4=",time.time()-start_4)
    print("loop_5 =", time.time()-start_5)
print(time.time()-start)

loop_1= 4.267692565917969e-05
loop_3= 0.0006208419799804688
loop_4= 0.000926971435546875
loop_1= 3.552436828613281e-05
loop_3= 0.00033593177795410156
loop_4= 0.0007534027099609375
loop_1= 2.1457672119140625e-05
loop_3= 8.487701416015625e-05
loop_4= 0.00014209747314453125
loop_1= 1.3828277587890625e-05
loop_3= 7.581710815429688e-05
loop_4= 0.0001323223114013672
loop_1= 1.2874603271484375e-05
loop_3= 6.961822509765625e-05
loop_4= 0.00012540817260742188
loop_1= 1.7404556274414062e-05
loop_3= 7.605552673339844e-05
loop_4= 0.00013256072998046875
loop_1= 2.1696090698242188e-05
loop_3= 8.034706115722656e-05
loop_4= 0.00013589859008789062
loop_1= 1.9311904907226562e-05
loop_3= 7.748603820800781e-05
loop_4= 0.00013494491577148438
loop_1= 1.3113021850585938e-05
loop_3= 0.00021839141845703125
loop_4= 0.00028896331787109375
loop_1= 1.4543533325195312e-05
loop_3= 7.367134094238281e-05
loop_4= 0.00012993812561035156
loop_1= 2.5987625122070312e-05
loop_3= 8.416175842285156e-05
loop_4= 0.0001399517059

loop_2= 0.002923727035522461
loop_1= 0.0032253265380859375
loop_1= 2.8371810913085938e-05
loop_1= 1.811981201171875e-05
loop_1= 1.5735626220703125e-05
yeah
loop_2= 0.0016374588012695312
loop_1= 0.0017766952514648438
loop_1= 3.0040740966796875e-05
yeah
loop_2= 0.0021092891693115234
loop_1= 0.0023415088653564453
yeah
loop_2= 0.0027883052825927734
loop_1= 0.002866506576538086
yeah
loop_2= 0.0027914047241210938
loop_1= 0.002904653549194336
yeah
loop_2= 0.003853321075439453
loop_1= 0.004065275192260742
yeah
loop_2= 0.0036063194274902344
loop_1= 0.0037631988525390625
yeah
loop_2= 0.001789093017578125
loop_1= 0.0018563270568847656
yeah
loop_2= 0.0014317035675048828
loop_1= 0.0018146038055419922
yeah
loop_2= 0.001546621322631836
loop_1= 0.0018184185028076172
yeah
loop_2= 0.0033097267150878906
loop_1= 0.0033996105194091797
yeah
loop_2= 0.0024404525756835938
loop_1= 0.002742290496826172
loop_3= 0.049787282943725586
loop_4= 0.04987311363220215
loop_1= 6.628036499023438e-05
loop_1= 2.8610229492187

yeah
loop_2= 0.06241917610168457
loop_1= 0.06255125999450684
loop_3= 0.09070110321044922
loop_4= 0.15483903884887695
loop_1= 0.00031495094299316406
loop_3= 0.000423431396484375
loop_1= 0.0001780986785888672
yeah
loop_2= 0.003904581069946289
loop_1= 0.004472017288208008
loop_1= 7.05718994140625e-05
loop_1= 0.0002148151397705078
yeah
loop_2= 0.011651992797851562
loop_1= 0.011792182922363281
loop_1= 4.9591064453125e-05
loop_1= 4.076957702636719e-05
loop_3= 0.01822638511657715
yeah
loop_2= 0.004033565521240234
loop_1= 0.0043337345123291016
yeah
loop_2= 0.00723719596862793
loop_1= 0.0073757171630859375
loop_1= 4.57763671875e-05
yeah
loop_2= 0.0021598339080810547
loop_1= 0.0022377967834472656
yeah
loop_2= 0.002156496047973633
loop_1= 0.002347707748413086
yeah
loop_2= 0.0030908584594726562
loop_1= 0.0032279491424560547
loop_1= 5.53131103515625e-05
yeah
loop_2= 0.0026597976684570312
loop_1= 0.002786874771118164
yeah
loop_2= 0.002312898635864258
loop_1= 0.0024118423461914062
yeah
loop_2= 0.0027

In [845]:
data_address_company_anonymized_df_2 = pd.DataFrame(data_address_company_anonymized_2,columns=column)

In [846]:
group_4 = []
tmp = list(data_address_company_anonymized_df_2["address_float_4"].unique())
tmp_1 = list(data_address_company_anonymized_df_2.address_float_4.values)
for n in group_3_backup:
    start = time.time()
    pgroup_4 = []
    for m in n:
        grou_4 = []
        for i in m:
            gro_4 = []
            for x,j in enumerate(i):
                gr_4 = []
                for l in tmp:
                    g_4 = []
                    for k in j:
                        if tmp_1[k] == l:
                            g_4.append(k)
                    if len(g_4) != 0:
                        gr_4.append(g_4)
                #print(gro_4)
                gro_4.append(gr_4)
            grou_4.append(gro_4)
        pgroup_4.append(grou_4)
    group_4.append(pgroup_4)
    print(time.time()-start)

0.036049604415893555
0.03357338905334473
0.0157315731048584
0.007137298583984375
0.12313628196716309
0.016802310943603516
0.0182952880859375
0.007981538772583008
0.011928558349609375
0.017156600952148438
0.06775999069213867
0.27748942375183105
0.8831973075866699
0.010660886764526367
0.0267179012298584
0.04378104209899902
0.051104068756103516
0.023730754852294922
0.04113960266113281
0.053395748138427734
0.00947713851928711
0.013064146041870117
0.009833335876464844
0.0803983211517334


In [847]:
group_4_backup = copy.deepcopy(group_4)

In [848]:
group_4_not_anonymized = []
for p,n in enumerate(group_4):
    for x,m in enumerate(n):
        for y,i in enumerate(m):
                for z,j in enumerate(i):
                    for k in j:
                        if 0<len(k)<20:
                            print(p,x,y,z)
                            print(len(k))
                            print(k)
                            group_4_not_anonymized.append([p,x,y,z,k])

4 0 1 3
15
[1119, 1138, 1376, 1400, 1587, 1819, 1841, 1873, 1894, 1918, 2077, 2274, 2300, 2311, 2399]
4 0 1 3
18
[1159, 1180, 1187, 1228, 1275, 1339, 1372, 1402, 1468, 1567, 1779, 1935, 2048, 2075, 2107, 2345, 2410, 2414]
4 0 1 3
1
[1198]
4 0 1 3
1
[1284]
4 0 1 3
1
[1780]
4 0 1 3
1
[1783]
4 0 1 3
1
[2238]
4 0 1 3
1
[2321]
4 0 1 3
1
[2362]
4 0 1 7
3
[1148, 1353, 1824]
4 0 1 7
7
[1178, 1369, 1638, 2017, 2335, 2388, 2404]
4 0 1 7
4
[1256, 1385, 1849, 2364]
4 0 1 7
5
[1274, 1418, 1466, 1933, 2042]
4 0 1 7
3
[1422, 1469, 1649]
4 0 1 7
1
[1859]
4 0 1 7
1
[2115]
4 0 1 7
1
[2141]
4 0 1 8
3
[1176, 1546, 1687]
4 0 1 8
18
[1189, 1332, 1342, 1391, 1447, 1580, 1607, 1621, 1630, 1811, 1822, 1860, 2035, 2082, 2172, 2377, 2415, 2426]
4 0 1 8
3
[1411, 1735, 1982]
4 0 1 8
4
[1419, 1549, 2363, 2376]
4 0 1 8
7
[1459, 1731, 2101, 2158, 2166, 2218, 2354]
4 0 2 3
7
[1124, 1132, 1569, 1787, 2012, 2230, 2236]
4 0 2 3
2
[1155, 2312]
4 0 2 3
5
[1177, 1519, 1689, 1943, 2250]
4 0 2 3
9
[1220, 1262, 1267, 1812, 185

12 2 0 2
2
[8603, 8749]
12 2 0 2
3
[8605, 8646, 8783]
12 2 0 2
2
[8607, 9042]
12 2 0 2
1
[8620]
12 2 0 2
1
[8628]
12 2 0 2
3
[8641, 8837, 9080]
12 2 0 2
2
[8647, 8953]
12 2 0 2
1
[8712]
12 2 0 2
1
[8714]
12 2 0 2
1
[8721]
12 2 0 2
1
[8750]
12 2 0 2
1
[8776]
12 2 0 2
1
[8779]
12 2 0 2
1
[8959]
12 2 0 2
1
[8962]
12 2 0 2
1
[8963]
12 2 0 2
1
[9015]
12 2 0 2
1
[9104]
12 2 0 4
3
[8549, 8892, 8988]
12 2 0 4
2
[8584, 8883]
12 2 0 4
5
[8586, 8627, 8708, 8784, 8895]
12 2 0 4
5
[8614, 8766, 8881, 8991, 9011]
12 2 0 4
2
[8723, 8820]
12 2 0 4
4
[8756, 8961, 9057, 9060]
12 2 1 1
3
[8631, 8654, 8836]
12 2 1 1
3
[8649, 8834, 8890]
12 2 1 1
5
[8726, 8972, 9008, 9009, 9079]
12 2 1 1
7
[8635, 8831, 8867, 8874, 8932, 8968, 9024]
12 2 1 1
4
[8527, 8673, 9012, 9018]
12 2 1 1
7
[8657, 8731, 8797, 8818, 8826, 9006, 9099]
12 2 1 1
2
[8695, 8715]
12 2 1 1
3
[8823, 9016, 9083]
12 3 0 4
5
[9118, 9306, 10078, 10316, 10336]
12 3 0 4
6
[9138, 9231, 9310, 10228, 10304, 10350]
12 3 0 4
4
[9151, 9440, 9595, 9649]
12 3

14 1 1 0
1
[16602]
14 1 1 0
1
[16613]
14 1 1 0
1
[16621]
14 1 1 0
2
[16637, 16643]
14 1 1 0
1
[16664]
21 0 0 1
1
[18909]
21 0 0 1
1
[18916]
21 0 0 1
1
[18924]
21 0 0 1
1
[18927]
21 0 0 1
1
[18928]
21 0 0 1
1
[18929]
21 0 0 1
1
[18935]
21 0 0 1
1
[18936]
21 0 0 1
1
[18940]
21 0 0 1
1
[18945]
21 0 0 1
1
[18953]
21 0 0 1
1
[18957]
21 0 0 1
1
[18959]
21 0 0 1
1
[18960]
21 0 0 1
2
[18965, 19000]
21 0 0 1
1
[18974]
21 0 0 1
1
[18975]
21 0 0 1
1
[18976]
21 0 0 1
1
[18982]
21 0 0 1
1
[18987]
21 0 0 1
1
[18988]
21 0 0 1
1
[18989]
21 0 0 1
1
[18993]
21 0 0 1
1
[19002]
21 0 0 1
1
[19009]
21 0 0 1
1
[19010]


In [849]:
tmp = data_address_company_anonymized_df_2.address_float_value_4
start_1 = time.time()
while len(group_4_not_anonymized) != 0:
    a = 0
    start = time.time()
#for i in group_4_not_anonymized:
    for x,j in enumerate(group_4_backup[group_4_not_anonymized[0][0]][group_4_not_anonymized[0][1]][group_4_not_anonymized[0][2]][group_4_not_anonymized[0][3]]):
        if len(j) != 0:
            if j[0] != group_4_not_anonymized[0][4][0]:
                if a == 0:
                    diff = abs(tmp[j[0]]-tmp[group_4_not_anonymized[0][4][0]])
                    closest_group = x
                    a += 1
                elif diff > abs(tmp[j[0]]-tmp[group_4_not_anonymized[0][4][0]]):
                    #print(diff)
                    #print(tmp[j[0]]-tmp[group_4_not_anonymized[0][1][0]])
                    diff = abs(tmp[j[0]]-tmp[group_4_not_anonymized[0][4][0]])
                    closest_group = x
    group_4_backup[group_4_not_anonymized[0][0]][group_4_not_anonymized[0][1]][group_4_not_anonymized[0][2]][group_4_not_anonymized[0][3]][closest_group].extend(group_4_not_anonymized[0][4])
    group_4_backup[group_4_not_anonymized[0][0]][group_4_not_anonymized[0][1]][group_4_not_anonymized[0][2]][group_4_not_anonymized[0][3]].remove(group_4_not_anonymized[0][4])
    group_4_not_anonymized = []
    for p,n in enumerate(group_4_backup):
        for x,m in enumerate(n):
            for y,i in enumerate(m):
                    for z,j in enumerate(i):
                        for k in j:
                            if 0<len(k)<20:
                                #print(p,x,y,z)
                                #print(len(k))
                                #print(k)
                                group_4_not_anonymized.append([p,x,y,z,k])
    if len(group_4_not_anonymized) != 0:
        print(group_4_not_anonymized[0])
    print(time.time()-start)
print(time.time()-start_1)

[4, 0, 1, 3, [1159, 1180, 1187, 1228, 1275, 1339, 1372, 1402, 1468, 1567, 1779, 1935, 2048, 2075, 2107, 2345, 2410, 2414]]
0.0038542747497558594
[4, 0, 1, 3, [1198]]
0.0014073848724365234
[4, 0, 1, 3, [1284, 1159, 1180, 1187, 1228, 1275, 1339, 1372, 1402, 1468, 1567, 1779, 1935, 2048, 2075, 2107, 2345, 2410, 2414]]
0.0010609626770019531
[4, 0, 1, 3, [1780]]
0.0009829998016357422
[4, 0, 1, 3, [2238, 1198]]
0.0012645721435546875
[4, 0, 1, 3, [2321, 1780, 2238, 1198]]
0.0010232925415039062
[4, 0, 1, 3, [2362, 1119, 1138, 1376, 1400, 1587, 1819, 1841, 1873, 1894, 1918, 2077, 2274, 2300, 2311, 2399]]
0.0009593963623046875
[4, 0, 1, 7, [1148, 1353, 1824]]
0.0007443428039550781
[4, 0, 1, 7, [1178, 1369, 1638, 2017, 2335, 2388, 2404, 1148, 1353, 1824]]
0.00098419189453125
[4, 0, 1, 7, [1256, 1385, 1849, 2364, 1178, 1369, 1638, 2017, 2335, 2388, 2404, 1148, 1353, 1824]]
0.0009567737579345703
[4, 0, 1, 7, [1274, 1418, 1466, 1933, 2042, 1256, 1385, 1849, 2364, 1178, 1369, 1638, 2017, 2335, 2388, 

[12, 2, 0, 2, [8750, 8590, 8555]]
0.0013735294342041016
[12, 2, 0, 2, [8776, 8712, 8647, 8953]]
0.0012006759643554688
[12, 2, 0, 2, [8779, 8603, 8749, 8594, 8857]]
0.0012714862823486328
[12, 2, 0, 2, [8959, 8644, 8847, 8721, 8605, 8646, 8783, 8698, 8998, 9110, 8785, 8607, 9042, 8558, 8812, 8750, 8590, 8555]]
0.0014660358428955078
[12, 2, 0, 2, [8963, 8714, 8592, 8624, 8702, 9103]]
0.0013349056243896484
[12, 2, 0, 2, [9015, 8540, 8581, 8639, 8720, 8824, 8958, 8779, 8603, 8749, 8594, 8857]]
0.0015521049499511719
[12, 2, 0, 4, [8549, 8892, 8988]]
0.0009317398071289062
[12, 2, 0, 4, [8584, 8883]]
0.0011508464813232422
[12, 2, 0, 4, [8586, 8627, 8708, 8784, 8895, 8584, 8883]]
0.0009033679962158203
[12, 2, 0, 4, [8614, 8766, 8881, 8991, 9011]]
0.0011012554168701172
[12, 2, 0, 4, [8723, 8820, 8586, 8627, 8708, 8784, 8895, 8584, 8883, 8614, 8766, 8881, 8991, 9011]]
0.0014219284057617188
[12, 2, 1, 1, [8631, 8654, 8836]]
0.0015170574188232422
[12, 2, 1, 1, [8649, 8834, 8890, 8631, 8654, 8836]]


[14, 0, 0, 0, [16258, 16283, 16304, 16340, 16341, 16343, 16353]]
0.0009717941284179688
[14, 0, 0, 0, [16263]]
0.0008969306945800781
[14, 0, 0, 0, [16285, 16299, 16322, 16240, 16264, 16297, 16309, 16316, 16325, 16351, 16352]]
0.0008130073547363281
[14, 0, 0, 0, [16311, 16365, 16263, 16285, 16299, 16322, 16240, 16264, 16297, 16309, 16316, 16325, 16351, 16352]]
0.0008008480072021484
[14, 0, 0, 0, [16347, 16242, 16313, 16258, 16283, 16304, 16340, 16341, 16343, 16353]]
0.0006387233734130859
[14, 0, 1, 0, [16241, 16245, 16256, 16271]]
0.0005459785461425781
[14, 0, 1, 0, [16269]]
0.0005974769592285156
[14, 0, 3, 0, [16246, 16276, 16323, 16328]]
0.000537872314453125
[14, 0, 3, 0, [16251, 16354]]
0.0009140968322753906
[14, 0, 3, 0, [16254]]
0.0009412765502929688
[14, 0, 3, 0, [16259, 16280, 16288, 16290, 16301, 16302, 16315, 16324, 16326, 16330, 16333, 16362, 16363]]
0.0007793903350830078
[14, 0, 3, 0, [16260, 16296, 16305, 16259, 16280, 16288, 16290, 16301, 16302, 16315, 16324, 16326, 16330, 1

In [850]:
start = time.time()
data_address_company_anonymized_3 = copy.deepcopy(data_address_company_anonymized_df_2.values)
column = list(data_address_company_anonymized_df_2.columns)
tmp = data_address_company_anonymized_df_2["address_float_4"].values
for v in group_4_backup:
    start_6 = time.time()
    for p in v:
        start_5 = time.time()
        for q in p:
            start_4 = time.time()
            for y in q:
                start_3 = time.time()
                for i in y:
                    a = 0
                    start_1 = time.time()
                    anonymized_data = []
                    for x,j in enumerate(i):
                        if x == 0:
                            fix = tmp[j]
                        elif fix != tmp[j]:
                            a+=1
                            for k in range(1,64):
                                if fix[0:-k] == tmp[j][0:-k]:
                                    anonymized_data.append(fix[0:-k] + "".join("0" for l in range(k)))
                                    break
                    if a > 0:
                        for x,anony in enumerate(anonymized_data):
                            if x == 0:
                                fix = anony
                            elif fix != anony:
                                for k in range(1,64):
                                    if fix[0:-k] == anony[0:-k]:
                                        fix = (fix[0:-k] + "".join("0" for l in range(k)))
                                        break
                        print("yeah")
                        column_num = column.index("address_float_4")
                        for m in i:
                            data_address_company_anonymized_3[m,column_num] = fix
                            for o in range(column_num+1,len(column)):
                                if o%2 == 1:
                                    data_address_company_anonymized_3[m,o] = "".join("0" for l in range(64))
                                else:
                                    data_address_company_anonymized_3[m,o] = np.float64(0)
                        print("loop_2=",time.time()-start_1)            
                    print("loop_1=",time.time()-start_1)
                print("loop_3=",time.time()-start_3)
            print("loop_4=",time.time()-start_4)
        print("loop_5 =", time.time()-start_5)
    print("loop_6 =",time.time()-start_6)
print(time.time()-start)

loop_1= 2.5510787963867188e-05
loop_3= 0.00013494491577148438
loop_4= 0.0003192424774169922
loop_5 = 0.0004723072052001953
loop_1= 1.9788742065429688e-05
loop_3= 7.772445678710938e-05
loop_4= 0.00013256072998046875
loop_5 = 0.000186920166015625
loop_1= 1.4066696166992188e-05
loop_3= 7.128715515136719e-05
loop_4= 0.00024890899658203125
loop_5 = 0.0003123283386230469
loop_1= 1.3828277587890625e-05
loop_3= 0.0004985332489013672
loop_4= 0.0005719661712646484
loop_5 = 0.0006470680236816406
loop_1= 2.2411346435546875e-05
loop_3= 0.00010752677917480469
loop_4= 0.00016307830810546875
loop_5 = 0.0002396106719970703
loop_1= 2.3126602172851562e-05
loop_3= 0.00039577484130859375
loop_4= 0.0004603862762451172
loop_5 = 0.0005159378051757812
loop_1= 2.1696090698242188e-05
loop_3= 8.0108642578125e-05
loop_4= 0.00013518333435058594
loop_5 = 0.0001895427703857422
loop_1= 1.8358230590820312e-05
loop_3= 7.557868957519531e-05
loop_4= 0.00013065338134765625
loop_5 = 0.00018596649169921875
loop_1= 1.19209289

loop_3= 0.00022602081298828125
loop_4= 0.00040984153747558594
yeah
loop_2= 0.0016391277313232422
loop_1= 0.0017490386962890625
loop_3= 0.0018219947814941406
loop_1= 2.1696090698242188e-05
loop_3= 0.00010228157043457031
loop_1= 1.811981201171875e-05
loop_3= 7.653236389160156e-05
loop_1= 2.1696090698242188e-05
loop_3= 7.915496826171875e-05
loop_1= 3.0517578125e-05
loop_3= 8.869171142578125e-05
loop_1= 1.3828277587890625e-05
loop_3= 7.104873657226562e-05
loop_1= 1.5497207641601562e-05
loop_3= 7.319450378417969e-05
loop_1= 2.4318695068359375e-05
loop_3= 0.00010657310485839844
loop_1= 2.8371810913085938e-05
loop_3= 8.678436279296875e-05
loop_1= 1.9788742065429688e-05
loop_3= 7.796287536621094e-05
loop_1= 1.621246337890625e-05
loop_3= 7.414817810058594e-05
loop_4= 0.003281831741333008
loop_1= 5.173683166503906e-05
loop_3= 0.00011086463928222656
loop_4= 0.00016570091247558594
yeah
loop_2= 0.002997875213623047
loop_1= 0.003066539764404297
loop_3= 0.0031223297119140625
loop_1= 2.980232238769531

loop_3= 0.00012040138244628906
loop_1= 3.9577484130859375e-05
loop_3= 0.0001316070556640625
loop_1= 2.7894973754882812e-05
loop_3= 0.000125885009765625
loop_1= 3.337860107421875e-05
loop_3= 0.0001289844512939453
loop_1= 2.574920654296875e-05
loop_3= 0.00010371208190917969
loop_1= 2.6702880859375e-05
loop_3= 9.918212890625e-05
loop_1= 1.7642974853515625e-05
loop_3= 0.0002994537353515625
loop_4= 0.06575369834899902
loop_1= 3.528594970703125e-05
loop_3= 0.0001392364501953125
loop_1= 2.6941299438476562e-05
loop_3= 0.00011205673217773438
loop_1= 4.029273986816406e-05
loop_3= 0.00011086463928222656
yeah
loop_2= 0.0033617019653320312
loop_1= 0.003620147705078125
loop_3= 0.003695964813232422
loop_1= 2.86102294921875e-05
loop_3= 0.00011014938354492188
loop_1= 2.2649765014648438e-05
loop_3= 9.989738464355469e-05
loop_4= 0.004858732223510742
loop_5 = 0.12049603462219238
yeah
loop_2= 0.0017027854919433594
loop_1= 0.0018742084503173828
loop_3= 0.001947164535522461
loop_1= 1.8358230590820312e-05
loo

loop_1= 2.7179718017578125e-05
loop_3= 0.00010824203491210938
loop_4= 0.07083797454833984
loop_5 = 0.07089376449584961
loop_6 = 0.07094883918762207
loop_1= 1.52587890625e-05
loop_3= 7.367134094238281e-05
loop_4= 0.00012350082397460938
loop_5 = 0.00017833709716796875
loop_1= 1.2636184692382812e-05
loop_3= 7.534027099609375e-05
loop_4= 0.00013136863708496094
loop_5 = 0.0001876354217529297
loop_1= 1.5974044799804688e-05
loop_3= 7.343292236328125e-05
loop_4= 0.0001327991485595703
loop_1= 1.0251998901367188e-05
loop_3= 6.771087646484375e-05
loop_4= 0.0001239776611328125
loop_5 = 0.0003657341003417969
loop_1= 2.7894973754882812e-05
loop_3= 8.559226989746094e-05
loop_4= 0.00014090538024902344
loop_5 = 0.00019550323486328125
loop_1= 1.9073486328125e-05
loop_3= 7.700920104980469e-05
loop_4= 0.0001327991485595703
loop_1= 1.1205673217773438e-05
loop_3= 6.866455078125e-05
loop_4= 0.0001246929168701172
loop_5 = 0.00036597251892089844
loop_1= 2.6226043701171875e-05
loop_3= 8.392333984375e-05
loop_4=

In [851]:
data_address_company_anonymized_df_3 = pd.DataFrame(data_address_company_anonymized_3,columns=column)

In [852]:
group_5 = []
tmp = list(data_address_company_anonymized_df_3["address_float_5"].unique())
tmp_1 = list(data_address_company_anonymized_df_3.address_float_5.values)
for q in group_4_backup:
    start = time.time()
    opgroup_5 = []
    for n in q:
        pgroup_5 = []
        for m in n:
            grou_5 = []
            for i in m:
                gro_5 = []
                for x,j in enumerate(i):
                    gr_5 = []
                    for l in tmp:
                        g_5 = []
                        for k in j:
                            if tmp_1[k] == l:
                                g_5.append(k)
                        if len(g_5) != 0:
                            gr_5.append(g_5)
                    #print(gro_5)
                    gro_5.append(gr_5)
                grou_5.append(gro_5)
            pgroup_5.append(grou_5)
        opgroup_5.append(pgroup_5)
    group_5.append(opgroup_5)
    print(time.time()-start)

0.02168583869934082
0.021000385284423828
0.011182546615600586
0.0071527957916259766
0.06955623626708984
0.008659124374389648
0.010174036026000977
0.004736423492431641
0.00641632080078125
0.011525154113769531
0.036595821380615234
0.15626239776611328
0.4699087142944336
0.0058062076568603516
0.017670154571533203
0.023439645767211914
0.025715112686157227
0.013608932495117188
0.02309393882751465
0.02474689483642578
0.004680156707763672
0.004747152328491211
0.004110097885131836
0.04094696044921875


In [853]:
group_5_backup = copy.deepcopy(group_5)

In [854]:
group_5_not_anonymized = []
for zz,jj in enumerate(group_5):
    for p,n in enumerate(jj):
        for x,m in enumerate(n):
            for y,i in enumerate(m):
                    for z,j in enumerate(i):
                        for k in j:
                            if 0<len(k)<20:
                                print(zz,p,x,y,z)
                                print(len(k))
                                print(k)
                                group_5_not_anonymized.append([zz,p,x,y,z,k])

4 0 0 3 0
8
[1219, 1270, 1487, 1500, 1583, 1721, 2094, 2453]
4 0 0 3 0
5
[1283, 1618, 1826, 1983, 2380]
4 0 0 3 0
1
[1599]
4 0 0 3 0
5
[1723, 1831, 2155, 2359, 2450]
4 0 0 3 0
1
[1738]
4 0 1 6 0
1
[1146]
4 0 1 6 0
4
[1258, 1358, 1503, 2371]
4 0 1 6 0
1
[1263]
4 0 1 6 0
4
[1297, 1698, 1857, 1907]
4 0 1 6 0
3
[1356, 1730, 1974]
4 0 1 6 0
3
[1366, 1773, 1962]
4 0 1 6 0
2
[1551, 1996]
4 0 1 6 0
6
[1942, 1997, 2113, 2225, 2248, 2449]
4 0 1 6 0
1
[2018]
4 0 1 6 0
1
[2114]
4 0 1 6 0
1
[2151]
4 0 1 6 0
1
[2389]
4 0 1 6 0
1
[2406]
4 0 2 5 0
11
[1151, 1191, 1248, 1365, 1388, 1489, 1650, 1799, 1919, 2309, 2365]
4 0 2 5 0
9
[1216, 1272, 1561, 1584, 1604, 2176, 2196, 2204, 2209]
4 0 2 5 0
9
[1316, 1387, 1947, 2129, 2145, 2241, 2366, 2405, 2448]
4 0 2 5 0
7
[1329, 1452, 1488, 1603, 1661, 1909, 2310]
4 0 2 5 0
8
[1355, 1476, 1722, 1832, 1987, 1991, 2186, 2219]
4 0 2 5 0
3
[1643, 1978, 2282]
4 0 2 5 0
2
[1692, 2221]
4 0 2 5 0
1
[2161]
11 3 0 5 0
13
[5476, 5509, 5528, 5589, 5641, 5701, 5706, 5767, 5793

In [855]:
group_5_backup = copy.deepcopy(group_5)

In [856]:
group_5_not_anonymized = []
for zz,jj in enumerate(group_5):
    for p,n in enumerate(jj):
        for x,m in enumerate(n):
            for y,i in enumerate(m):
                    for z,j in enumerate(i):
                        for k in j:
                            if 0<len(k)<20:
                                print(zz,p,x,y,z)
                                print(len(k))
                                print(k)
                                group_5_not_anonymized.append([zz,p,x,y,z,k])

4 0 0 3 0
8
[1219, 1270, 1487, 1500, 1583, 1721, 2094, 2453]
4 0 0 3 0
5
[1283, 1618, 1826, 1983, 2380]
4 0 0 3 0
1
[1599]
4 0 0 3 0
5
[1723, 1831, 2155, 2359, 2450]
4 0 0 3 0
1
[1738]
4 0 1 6 0
1
[1146]
4 0 1 6 0
4
[1258, 1358, 1503, 2371]
4 0 1 6 0
1
[1263]
4 0 1 6 0
4
[1297, 1698, 1857, 1907]
4 0 1 6 0
3
[1356, 1730, 1974]
4 0 1 6 0
3
[1366, 1773, 1962]
4 0 1 6 0
2
[1551, 1996]
4 0 1 6 0
6
[1942, 1997, 2113, 2225, 2248, 2449]
4 0 1 6 0
1
[2018]
4 0 1 6 0
1
[2114]
4 0 1 6 0
1
[2151]
4 0 1 6 0
1
[2389]
4 0 1 6 0
1
[2406]
4 0 2 5 0
11
[1151, 1191, 1248, 1365, 1388, 1489, 1650, 1799, 1919, 2309, 2365]
4 0 2 5 0
9
[1216, 1272, 1561, 1584, 1604, 2176, 2196, 2204, 2209]
4 0 2 5 0
9
[1316, 1387, 1947, 2129, 2145, 2241, 2366, 2405, 2448]
4 0 2 5 0
7
[1329, 1452, 1488, 1603, 1661, 1909, 2310]
4 0 2 5 0
8
[1355, 1476, 1722, 1832, 1987, 1991, 2186, 2219]
4 0 2 5 0
3
[1643, 1978, 2282]
4 0 2 5 0
2
[1692, 2221]
4 0 2 5 0
1
[2161]
11 3 0 5 0
13
[5476, 5509, 5528, 5589, 5641, 5701, 5706, 5767, 5793

In [857]:
tmp = data_address_company_anonymized_df_3.address_float_value_5
start_1 = time.time()
while len(group_5_not_anonymized) != 0:
    a = 0
    start = time.time()
#for i in group_5_not_anonymized:
    for x,j in enumerate(group_5_backup[group_5_not_anonymized[0][0]][group_5_not_anonymized[0][1]][group_5_not_anonymized[0][2]][group_5_not_anonymized[0][3]][group_5_not_anonymized[0][4]]):
        if len(j) != 0:
            if j[0] != group_5_not_anonymized[0][5][0]:
                if a == 0:
                    diff = abs(tmp[j[0]]-tmp[group_5_not_anonymized[0][5][0]])
                    closest_group = x
                    a += 1
                elif diff > abs(tmp[j[0]]-tmp[group_5_not_anonymized[0][5][0]]):
                    #print(diff)
                    #print(tmp[j[0]]-tmp[group_5_not_anonymized[0][1][0]])
                    diff = abs(tmp[j[0]]-tmp[group_5_not_anonymized[0][5][0]])
                    closest_group = x
    group_5_backup[group_5_not_anonymized[0][0]][group_5_not_anonymized[0][1]][group_5_not_anonymized[0][2]][group_5_not_anonymized[0][3]][group_5_not_anonymized[0][4]][closest_group].extend(group_5_not_anonymized[0][5])
    group_5_backup[group_5_not_anonymized[0][0]][group_5_not_anonymized[0][1]][group_5_not_anonymized[0][2]][group_5_not_anonymized[0][3]][group_5_not_anonymized[0][4]].remove(group_5_not_anonymized[0][5])
    group_5_not_anonymized = []
    for zz,jj in enumerate(group_5_backup):
        for p,n in enumerate(jj):
            for x,m in enumerate(n):
                for y,i in enumerate(m):
                        for z,j in enumerate(i):
                            for k in j:
                                if 0<len(k)<20:
                                    print(zz,p,x,y,z)
                                    print(len(k))
                                    print(k)
                                    group_5_not_anonymized.append([zz,p,x,y,z,k])
    if len(group_5_not_anonymized) != 0:
        print(group_5_not_anonymized[0])
    print(time.time()-start)
print(time.time()-start_1)

4 0 0 3 0
13
[1283, 1618, 1826, 1983, 2380, 1219, 1270, 1487, 1500, 1583, 1721, 2094, 2453]
4 0 0 3 0
1
[1599]
4 0 0 3 0
5
[1723, 1831, 2155, 2359, 2450]
4 0 0 3 0
1
[1738]
4 0 1 6 0
1
[1146]
4 0 1 6 0
4
[1258, 1358, 1503, 2371]
4 0 1 6 0
1
[1263]
4 0 1 6 0
4
[1297, 1698, 1857, 1907]
4 0 1 6 0
3
[1356, 1730, 1974]
4 0 1 6 0
3
[1366, 1773, 1962]
4 0 1 6 0
2
[1551, 1996]
4 0 1 6 0
6
[1942, 1997, 2113, 2225, 2248, 2449]
4 0 1 6 0
1
[2018]
4 0 1 6 0
1
[2114]
4 0 1 6 0
1
[2151]
4 0 1 6 0
1
[2389]
4 0 1 6 0
1
[2406]
4 0 2 5 0
11
[1151, 1191, 1248, 1365, 1388, 1489, 1650, 1799, 1919, 2309, 2365]
4 0 2 5 0
9
[1216, 1272, 1561, 1584, 1604, 2176, 2196, 2204, 2209]
4 0 2 5 0
9
[1316, 1387, 1947, 2129, 2145, 2241, 2366, 2405, 2448]
4 0 2 5 0
7
[1329, 1452, 1488, 1603, 1661, 1909, 2310]
4 0 2 5 0
8
[1355, 1476, 1722, 1832, 1987, 1991, 2186, 2219]
4 0 2 5 0
3
[1643, 1978, 2282]
4 0 2 5 0
2
[1692, 2221]
4 0 2 5 0
1
[2161]
11 3 0 5 0
13
[5476, 5509, 5528, 5589, 5641, 5701, 5706, 5767, 5793, 5816, 5822

12 2 0 0 0
1
[8588]
12 2 0 0 0
1
[8606]
12 2 0 0 0
1
[8623]
12 2 0 0 0
1
[8630]
12 2 0 0 0
1
[8642]
12 2 0 0 0
1
[8672]
12 2 0 0 0
1
[8684]
12 2 0 0 0
1
[8692]
12 2 0 0 0
1
[8693]
12 2 0 0 0
1
[8770]
12 2 0 0 0
1
[8795]
12 2 0 0 0
1
[8800]
12 2 0 0 0
1
[8822]
12 2 0 0 0
1
[8870]
12 2 0 0 0
1
[8876]
12 2 0 0 0
1
[8896]
12 2 0 0 0
1
[8897]
12 2 0 0 0
2
[8903, 8975]
12 2 0 0 0
1
[8936]
12 2 0 0 0
1
[8944]
12 2 0 0 0
1
[8947]
12 2 0 0 0
1
[8956]
12 2 0 0 0
1
[8993]
12 2 0 0 0
1
[9020]
12 2 0 0 0
1
[9047]
12 2 0 0 0
1
[9058]
12 2 0 0 0
1
[9101]
12 3 0 5 0
6
[9168, 9190, 9581, 9643, 10021, 10217]
12 3 0 5 0
4
[9246, 9756, 9977, 10107]
12 3 0 5 0
15
[9281, 9428, 9475, 9505, 9528, 9566, 9625, 9650, 9655, 9796, 10044, 10087, 10255, 10464, 10574]
12 3 0 5 0
7
[9367, 9761, 9801, 9923, 10153, 10417, 10512]
12 3 0 5 0
7
[9376, 9529, 9951, 10056, 10400, 10438, 10491]
12 3 0 5 0
1
[9836]
12 3 0 6 0
2
[9122, 9672]
12 3 0 6 0
1
[9167]
12 3 0 6 0
1
[9186]
12 3 0 6 0
1
[9203]
12 3 0 6 0
1
[9263]
12 3 0 6

1
[9101]
12 3 0 5 0
6
[9168, 9190, 9581, 9643, 10021, 10217]
12 3 0 5 0
4
[9246, 9756, 9977, 10107]
12 3 0 5 0
15
[9281, 9428, 9475, 9505, 9528, 9566, 9625, 9650, 9655, 9796, 10044, 10087, 10255, 10464, 10574]
12 3 0 5 0
7
[9367, 9761, 9801, 9923, 10153, 10417, 10512]
12 3 0 5 0
7
[9376, 9529, 9951, 10056, 10400, 10438, 10491]
12 3 0 5 0
1
[9836]
12 3 0 6 0
2
[9122, 9672]
12 3 0 6 0
1
[9167]
12 3 0 6 0
1
[9186]
12 3 0 6 0
1
[9203]
12 3 0 6 0
1
[9263]
12 3 0 6 0
1
[9266]
12 3 0 6 0
1
[9410]
12 3 0 6 0
2
[9415, 10489]
12 3 0 6 0
1
[9458]
12 3 0 6 0
1
[9485]
12 3 0 6 0
1
[9494]
12 3 0 6 0
1
[9582]
12 3 0 6 0
1
[9667]
12 3 0 6 0
1
[9752]
12 3 0 6 0
1
[9790]
12 3 0 6 0
1
[9840]
12 3 0 6 0
1
[9901]
12 3 0 6 0
2
[9912, 9952]
12 3 0 6 0
1
[9962]
12 3 0 6 0
1
[9989]
12 3 0 6 0
1
[9994]
12 3 0 6 0
1
[10074]
12 3 0 6 0
1
[10162]
12 3 0 6 0
1
[10227]
12 3 0 6 0
1
[10229]
12 3 0 6 0
1
[10241]
12 3 0 6 0
1
[10246]
12 3 0 6 0
1
[10253]
12 3 0 6 0
1
[10282]
12 3 0 6 0
1
[10326]
12 3 0 6 0
1
[10496]
12

[10074]
12 3 0 6 0
1
[10162]
12 3 0 6 0
1
[10227]
12 3 0 6 0
1
[10229]
12 3 0 6 0
1
[10241]
12 3 0 6 0
1
[10246]
12 3 0 6 0
1
[10253]
12 3 0 6 0
1
[10282]
12 3 0 6 0
1
[10326]
12 3 0 6 0
1
[10496]
12 3 0 15 0
2
[9366, 10383]
12 3 0 15 0
3
[9370, 9611, 10201]
12 3 0 15 0
1
[9374]
12 3 0 15 0
2
[9404, 9478]
12 3 0 15 0
2
[9507, 9570]
12 3 0 15 0
1
[9589]
12 3 0 15 0
1
[9639]
12 3 0 15 0
2
[9759, 9787]
12 3 0 15 0
2
[9783, 10583]
12 3 0 15 0
3
[9802, 9916, 10479]
12 3 0 15 0
4
[9819, 9830, 10220, 10592]
12 3 0 15 0
1
[10109]
12 3 0 15 0
1
[10268]
12 3 0 15 0
1
[10280]
12 3 0 15 0
1
[10497]
12 3 0 15 0
1
[10547]
12 3 0 15 0
1
[10550]
12 3 1 5 0
7
[9580, 9674, 9865, 10431, 10461, 10490, 10591]
12 3 1 5 0
11
[9204, 9222, 9324, 9369, 9778, 9780, 9816, 10293, 10413, 10546, 10589]
12 3 1 5 0
9
[9425, 9454, 9521, 9596, 9681, 10009, 10140, 10147, 10368]
12 3 1 5 0
3
[9386, 10294, 10396]
12 4 0 0 1
2
[10597, 11641]
12 4 0 0 1
3
[10609, 10772, 11016]
12 4 0 0 1
5
[10709, 10742, 11654, 12259, 12792]

12 7 1 0 0
1
[15123]
12 7 1 0 0
1
[15129]
12 7 1 0 0
1
[15153]
12 7 1 0 0
1
[15276]
12 7 1 0 0
2
[15294, 15761]
12 7 1 0 0
2
[15313, 16053]
12 7 1 0 0
5
[15343, 15585, 15787, 15831, 16045]
12 7 1 0 0
2
[15367, 15891]
12 7 1 0 0
2
[15403, 15457]
12 7 1 0 0
1
[15429]
12 7 1 0 0
1
[15507]
12 7 1 0 0
1
[15551]
12 7 1 0 0
1
[15579]
12 7 1 0 0
1
[15598]
12 7 1 0 0
1
[15705]
12 7 1 0 0
1
[15740]
12 7 1 0 0
1
[15757]
12 7 1 0 0
1
[15890]
12 7 1 0 0
1
[16110]
12 7 1 0 0
1
[16111]
12 7 1 3 0
1
[15076]
12 7 1 3 0
1
[15249]
12 7 1 3 0
1
[15293]
12 7 1 3 0
1
[15299]
12 7 1 3 0
1
[15308]
12 7 1 3 0
1
[15364]
12 7 1 3 0
1
[15380]
12 7 1 3 0
1
[15392]
12 7 1 3 0
1
[15452]
12 7 1 3 0
1
[15455]
12 7 1 3 0
1
[15484]
12 7 1 3 0
1
[15492]
12 7 1 3 0
1
[15557]
12 7 1 3 0
1
[15571]
12 7 1 3 0
1
[15600]
12 7 1 3 0
1
[15677]
12 7 1 3 0
2
[15681, 15808]
12 7 1 3 0
1
[15687]
12 7 1 3 0
1
[15695]
12 7 1 3 0
1
[15706]
12 7 1 3 0
1
[15778]
12 7 1 3 0
1
[15834]
12 7 1 3 0
1
[15853]
12 7 1 3 0
1
[15899]
12 7 1 3 0
1


...

1
[15492]
12 7 1 3 0
1
[15557]
12 7 1 3 0
1
[15571]
12 7 1 3 0
1
[15600]
12 7 1 3 0
1
[15677]
12 7 1 3 0
2
[15681, 15808]
12 7 1 3 0
1
[15687]
12 7 1 3 0
1
[15695]
12 7 1 3 0
1
[15706]
12 7 1 3 0
1
[15778]
12 7 1 3 0
1
[15834]
12 7 1 3 0
1
[15853]
12 7 1 3 0
1
[15899]
12 7 1 3 0
1
[15901]
12 7 1 3 0
1
[15995]
12 7 1 3 0
1
[16057]
12 7 1 3 0
1
[16067]
12 7 1 3 0
1
[16101]
12 7 1 3 0
1
[16112]
12 7 1 3 0
1
[16128]
12 7 1 3 0
1
[16129]
12 7 1 3 0
1
[16130]
14 0 4 0 0
16
[16266, 16268, 16274, 16291, 16292, 16312, 16320, 16336, 16337, 16339, 16344, 16345, 16346, 16359, 16361, 16367]
14 0 4 0 0
1
[16270]
14 0 4 0 0
1
[16279]
14 0 4 0 0
1
[16284]
14 0 4 0 0
2
[16321, 16358]
14 0 4 0 0
1
[16338]
14 1 2 0 0
1
[16541]
14 1 2 0 0
1
[16546]
14 1 2 0 0
1
[16572]
14 1 2 0 0
17
[16574, 16579, 16582, 16586, 16590, 16597, 16601, 16610, 16612, 16614, 16615, 16624, 16626, 16655, 16660, 16666, 16670]
14 1 2 0 0
1
[16629]
14 1 2 0 0
1
[16638]
[12, 7, 1, 0, 0, [15740, 15429, 15276]]
0.08276224136352539
12 7

12 7 1 3 0
1
[15706]
12 7 1 3 0
2
[15778, 15076]
12 7 1 3 0
1
[15834]
12 7 1 3 0
1
[15853]
12 7 1 3 0
1
[15899]
12 7 1 3 0
1
[15901]
12 7 1 3 0
1
[15995]
12 7 1 3 0
1
[16057]
12 7 1 3 0
1
[16067]
12 7 1 3 0
1
[16101]
12 7 1 3 0
1
[16112]
12 7 1 3 0
1
[16128]
12 7 1 3 0
1
[16129]
12 7 1 3 0
1
[16130]
14 0 4 0 0
16
[16266, 16268, 16274, 16291, 16292, 16312, 16320, 16336, 16337, 16339, 16344, 16345, 16346, 16359, 16361, 16367]
14 0 4 0 0
1
[16270]
14 0 4 0 0
1
[16279]
14 0 4 0 0
1
[16284]
14 0 4 0 0
2
[16321, 16358]
14 0 4 0 0
1
[16338]
14 1 2 0 0
1
[16541]
14 1 2 0 0
1
[16546]
14 1 2 0 0
1
[16572]
14 1 2 0 0
17
[16574, 16579, 16582, 16586, 16590, 16597, 16601, 16610, 16612, 16614, 16615, 16624, 16626, 16655, 16660, 16666, 16670]
14 1 2 0 0
1
[16629]
14 1 2 0 0
1
[16638]
[12, 7, 1, 3, 0, [15293]]
0.07392048835754395
12 7 1 3 0
1
[15299]
12 7 1 3 0
1
[15308]
12 7 1 3 0
1
[15364]
12 7 1 3 0
1
[15380]
12 7 1 3 0
1
[15392]
12 7 1 3 0
1
[15452]
12 7 1 3 0
1
[15455]
12 7 1 3 0
1
[15484]
12 7 1 

[16129]
12 7 1 3 0
1
[16130]
14 0 4 0 0
16
[16266, 16268, 16274, 16291, 16292, 16312, 16320, 16336, 16337, 16339, 16344, 16345, 16346, 16359, 16361, 16367]
14 0 4 0 0
1
[16270]
14 0 4 0 0
1
[16279]
14 0 4 0 0
1
[16284]
14 0 4 0 0
2
[16321, 16358]
14 0 4 0 0
1
[16338]
14 1 2 0 0
1
[16541]
14 1 2 0 0
1
[16546]
14 1 2 0 0
1
[16572]
14 1 2 0 0
17
[16574, 16579, 16582, 16586, 16590, 16597, 16601, 16610, 16612, 16614, 16615, 16624, 16626, 16655, 16660, 16666, 16670]
14 1 2 0 0
1
[16629]
14 1 2 0 0
1
[16638]
[12, 7, 1, 3, 0, [15492, 15452]]
0.07535123825073242
12 7 1 3 0
1
[15557]
12 7 1 3 0
2
[15571, 15299]
12 7 1 3 0
1
[15600]
12 7 1 3 0
3
[15677, 15249, 15293]
12 7 1 3 0
2
[15681, 15808]
12 7 1 3 0
1
[15687]
12 7 1 3 0
1
[15695]
12 7 1 3 0
1
[15706]
12 7 1 3 0
4
[15778, 15076, 15392, 15364]
12 7 1 3 0
3
[15834, 15380, 15455]
12 7 1 3 0
1
[15853]
12 7 1 3 0
2
[15899, 15308]
12 7 1 3 0
1
[15901]
12 7 1 3 0
3
[15995, 15492, 15452]
12 7 1 3 0
1
[16057]
12 7 1 3 0
2
[16067, 15484]
12 7 1 3 0
1


12 7 1 3 0
1
[16112]
12 7 1 3 0
1
[16128]
12 7 1 3 0
2
[16129, 15600]
12 7 1 3 0
7
[16130, 15571, 15299, 15778, 15076, 15392, 15364]
14 0 4 0 0
16
[16266, 16268, 16274, 16291, 16292, 16312, 16320, 16336, 16337, 16339, 16344, 16345, 16346, 16359, 16361, 16367]
14 0 4 0 0
1
[16270]
14 0 4 0 0
1
[16279]
14 0 4 0 0
1
[16284]
14 0 4 0 0
2
[16321, 16358]
14 0 4 0 0
1
[16338]
14 1 2 0 0
1
[16541]
14 1 2 0 0
1
[16546]
14 1 2 0 0
1
[16572]
14 1 2 0 0
17
[16574, 16579, 16582, 16586, 16590, 16597, 16601, 16610, 16612, 16614, 16615, 16624, 16626, 16655, 16660, 16666, 16670]
14 1 2 0 0
1
[16629]
14 1 2 0 0
1
[16638]
[12, 7, 1, 3, 0, [15834, 15380, 15455]]
0.07885169982910156
12 7 1 3 0
1
[15853]
12 7 1 3 0
5
[15899, 15308, 15677, 15249, 15293]
12 7 1 3 0
2
[15901, 15706]
12 7 1 3 0
10
[15995, 15492, 15452, 15687, 15557, 15681, 15808, 15834, 15380, 15455]
12 7 1 3 0
1
[16057]
12 7 1 3 0
3
[16067, 15484, 15695]
12 7 1 3 0
1
[16101]
12 7 1 3 0
1
[16112]
12 7 1 3 0
1
[16128]
12 7 1 3 0
2
[16129, 15600]

14 1 2 0 0
1
[16629]
14 1 2 0 0
1
[16638]
[14, 0, 4, 0, 0, [16266, 16268, 16274, 16291, 16292, 16312, 16320, 16336, 16337, 16339, 16344, 16345, 16346, 16359, 16361, 16367]]
0.06588077545166016
14 0 4 0 0
1
[16270]
14 0 4 0 0
1
[16279]
14 0 4 0 0
1
[16284]
14 0 4 0 0
2
[16321, 16358]
14 0 4 0 0
17
[16338, 16266, 16268, 16274, 16291, 16292, 16312, 16320, 16336, 16337, 16339, 16344, 16345, 16346, 16359, 16361, 16367]
14 1 2 0 0
1
[16541]
14 1 2 0 0
1
[16546]
14 1 2 0 0
1
[16572]
14 1 2 0 0
17
[16574, 16579, 16582, 16586, 16590, 16597, 16601, 16610, 16612, 16614, 16615, 16624, 16626, 16655, 16660, 16666, 16670]
14 1 2 0 0
1
[16629]
14 1 2 0 0
1
[16638]
[14, 0, 4, 0, 0, [16270]]
0.003692626953125
14 0 4 0 0
1
[16279]
14 0 4 0 0
2
[16284, 16270]
14 0 4 0 0
2
[16321, 16358]
14 0 4 0 0
17
[16338, 16266, 16268, 16274, 16291, 16292, 16312, 16320, 16336, 16337, 16339, 16344, 16345, 16346, 16359, 16361, 16367]
14 1 2 0 0
1
[16541]
14 1 2 0 0
1
[16546]
14 1 2 0 0
1
[16572]
14 1 2 0 0
17
[16574, 165

In [858]:
start = time.time()
data_address_company_anonymized_4 = copy.deepcopy(data_address_company_anonymized_df_3.values)
column = list(data_address_company_anonymized_df_3.columns)
tmp = data_address_company_anonymized_df_3["address_float_5"].values
for aa in group_5_backup:
    for v in aa:
        start_6 = time.time()
        for p in v:
            start_5 = time.time()
            for q in p:
                start_5 = time.time()
                for y in q:
                    start_3 = time.time()
                    for i in y:
                        a = 0
                        start_1 = time.time()
                        anonymized_data = []
                        for x,j in enumerate(i):
                            if x == 0:
                                fix = tmp[j]
                            elif fix != tmp[j]:
                                a+=1
                                for k in range(1,64):
                                    if fix[0:-k] == tmp[j][0:-k]:
                                        anonymized_data.append(fix[0:-k] + "".join("0" for l in range(k)))
                                        break
                        if a > 0:
                            for x,anony in enumerate(anonymized_data):
                                if x == 0:
                                    fix = anony
                                elif fix != anony:
                                    for k in range(1,64):
                                        if fix[0:-k] == anony[0:-k]:
                                            fix = (fix[0:-k] + "".join("0" for l in range(k)))
                                            break
                            #print("yeah")
                            column_num = column.index("address_float_5")
                            for m in i:
                                data_address_company_anonymized_4[m,column_num] = fix
                                for o in range(column_num+1,len(column)):
                                    if o%2 == 1:
                                        data_address_company_anonymized_4[m,o] = "".join("0" for l in range(64))
                                    else:
                                        data_address_company_anonymized_4[m,o] = np.float64(0)
                            #print("loop_2=",time.time()-start_1)            
                        #print("loop_1=",time.time()-start_1)
                    #print("loop_3=",time.time()-start_3)
                #print("loop_5=",time.time()-start_5)
            #print("loop_5 =", time.time()-start_5)
        #print("loop_6 =",time.time()-start_6)
    #print(time.time()-start)

In [859]:
data_address_company_anonymized_df_4 = pd.DataFrame(data_address_company_anonymized_4,columns = column)

In [860]:
group_6 = []
tmp = list(data_address_company_anonymized_df_4["address_float_6"].unique())
tmp_1 = list(data_address_company_anonymized_df_4.address_float_6.values)
for aa in group_5_backup:
    uopgroup_6 = []
    for q in aa:
        start = time.time()
        opgroup_6 = []
        for n in q:
            pgroup_6 = []
            for m in n:
                grou_6 = []
                for i in m:
                    gro_6 = []
                    for x,j in enumerate(i):
                        gr_6 = []
                        for l in tmp:
                            g_6 = []
                            for k in j:
                                if tmp_1[k] == l:
                                    g_6.append(k)
                            if len(g_6) != 0:
                                gr_6.append(g_6)
                        #print(gro_6)
                        gro_6.append(gr_6)
                    grou_6.append(gro_6)
                pgroup_6.append(grou_6)
            opgroup_6.append(pgroup_6)
        uopgroup_6.append(opgroup_6)
    group_6.append(uopgroup_6)
        #print(time.time()-start)

In [861]:
group_6_backup = copy.deepcopy(group_6)

In [862]:
group_6_not_anonymized = []
for yy,ii in enumerate(group_6):
    for zz,jj in enumerate(ii):
        for p,n in enumerate(jj):
            for x,m in enumerate(n):
                for y,i in enumerate(m):
                        for z,j in enumerate(i):
                            for k in j:
                                if 0<len(k)<20:
                                    print(yy,zz,p,x,y,z)
                                    print(len(k))
                                    print(k)
                                    group_6_not_anonymized.append([yy,zz,p,x,y,z,k])

4 0 1 6 0 0
8
[1195, 1227, 1259, 1298, 1319, 1672, 1804, 2423]
4 0 1 6 0 0
8
[1196, 1273, 1289, 1375, 1573, 1579, 1908, 2328]
4 0 1 6 0 0
6
[1286, 1478, 1755, 1768, 2078, 2157]
4 0 1 6 0 0
5
[1360, 1406, 1517, 1797, 2034]
4 0 1 6 0 0
7
[1581, 1589, 1866, 2083, 2123, 2257, 2409]
4 0 1 6 0 0
4
[1586, 1627, 2327, 2407]
4 0 1 6 0 0
2
[1745, 1975]
4 0 1 6 0 0
4
[1807, 2060, 2400, 2454]
4 0 1 6 0 0
1
[1886]
12 4 0 0 5 0
2
[10610, 12572]
12 4 0 0 5 0
1
[10690]
12 4 0 0 5 0
1
[10705]
12 4 0 0 5 0
2
[10719, 10754]
12 4 0 0 5 0
1
[10731]
12 4 0 0 5 0
1
[10757]
12 4 0 0 5 0
1
[10880]
12 4 0 0 5 0
4
[10940, 11312, 11686, 12538]
12 4 0 0 5 0
2
[11011, 11090]
12 4 0 0 5 0
1
[11152]
12 4 0 0 5 0
1
[11160]
12 4 0 0 5 0
1
[11258]
12 4 0 0 5 0
1
[11405]
12 4 0 0 5 0
1
[11464]
12 4 0 0 5 0
2
[11503, 12387]
12 4 0 0 5 0
3
[11575, 11651, 11938]
12 4 0 0 5 0
2
[11638, 12833]
12 4 0 0 5 0
1
[11735]
12 4 0 0 5 0
1
[11740]
12 4 0 0 5 0
2
[11747, 12722]
12 4 0 0 5 0
1
[11762]
12 4 0 0 5 0
1
[11800]
12 4 0 0 5 0

In [863]:
tmp = data_address_company_anonymized_df_4.address_float_value_6
start_1 = time.time()
while len(group_6_not_anonymized) != 0:
    a = 0
    start = time.time()
#for i in group_6_not_anonymized:
    for x,j in enumerate(group_6_backup[group_6_not_anonymized[0][0]][group_6_not_anonymized[0][1]][group_6_not_anonymized[0][2]][group_6_not_anonymized[0][3]][group_6_not_anonymized[0][4]][group_6_not_anonymized[0][5]]):
        if len(j) != 0:
            if j[0] != group_6_not_anonymized[0][6][0]:
                if a == 0:
                    diff = abs(tmp[j[0]]-tmp[group_6_not_anonymized[0][6][0]])
                    closest_group = x
                    a += 1
                elif diff > abs(tmp[j[0]]-tmp[group_6_not_anonymized[0][6][0]]):
                    ##print(diff)
                    ##print(tmp[j[0]]-tmp[group_6_not_anonymized[0][1][0]])
                    diff = abs(tmp[j[0]]-tmp[group_6_not_anonymized[0][6][0]])
                    closest_group = x
    group_6_backup[group_6_not_anonymized[0][0]][group_6_not_anonymized[0][1]][group_6_not_anonymized[0][2]][group_6_not_anonymized[0][3]][group_6_not_anonymized[0][4]][group_6_not_anonymized[0][5]][closest_group].extend(group_6_not_anonymized[0][6])
    group_6_backup[group_6_not_anonymized[0][0]][group_6_not_anonymized[0][1]][group_6_not_anonymized[0][2]][group_6_not_anonymized[0][3]][group_6_not_anonymized[0][4]][group_6_not_anonymized[0][5]].remove(group_6_not_anonymized[0][6])
    group_6_not_anonymized = []
    for yy,ii in enumerate(group_6_backup):
        for zz,jj in enumerate(ii):
            for p,n in enumerate(jj):
                for x,m in enumerate(n):
                    for y,i in enumerate(m):
                            for z,j in enumerate(i):
                                for k in j:
                                    if 0<len(k)<20:
                                        print(yy,zz,p,x,y,z)
                                        print(len(k))
                                        print(k)
                                        group_6_not_anonymized.append([yy,zz,p,x,y,z,k])
    if len(group_6_not_anonymized) != 0:
        print(group_6_not_anonymized[0])
    #print(time.time()-start)
print(time.time()-start_1)

4 0 1 6 0 0
8
[1196, 1273, 1289, 1375, 1573, 1579, 1908, 2328]
4 0 1 6 0 0
6
[1286, 1478, 1755, 1768, 2078, 2157]
4 0 1 6 0 0
5
[1360, 1406, 1517, 1797, 2034]
4 0 1 6 0 0
7
[1581, 1589, 1866, 2083, 2123, 2257, 2409]
4 0 1 6 0 0
4
[1586, 1627, 2327, 2407]
4 0 1 6 0 0
2
[1745, 1975]
4 0 1 6 0 0
12
[1807, 2060, 2400, 2454, 1195, 1227, 1259, 1298, 1319, 1672, 1804, 2423]
4 0 1 6 0 0
1
[1886]
12 4 0 0 5 0
2
[10610, 12572]
12 4 0 0 5 0
1
[10690]
12 4 0 0 5 0
1
[10705]
12 4 0 0 5 0
2
[10719, 10754]
12 4 0 0 5 0
1
[10731]
12 4 0 0 5 0
1
[10757]
12 4 0 0 5 0
1
[10880]
12 4 0 0 5 0
4
[10940, 11312, 11686, 12538]
12 4 0 0 5 0
2
[11011, 11090]
12 4 0 0 5 0
1
[11152]
12 4 0 0 5 0
1
[11160]
12 4 0 0 5 0
1
[11258]
12 4 0 0 5 0
1
[11405]
12 4 0 0 5 0
1
[11464]
12 4 0 0 5 0
2
[11503, 12387]
12 4 0 0 5 0
3
[11575, 11651, 11938]
12 4 0 0 5 0
2
[11638, 12833]
12 4 0 0 5 0
1
[11735]
12 4 0 0 5 0
1
[11740]
12 4 0 0 5 0
2
[11747, 12722]
12 4 0 0 5 0
1
[11762]
12 4 0 0 5 0
1
[11800]
12 4 0 0 5 0
1
[11949]
12 

[11385]
12 4 1 0 4 0
1
[11603]
12 4 1 0 4 0
1
[11653]
12 4 1 0 4 0
1
[11681]
12 4 1 0 4 0
1
[11712]
12 4 1 0 4 0
1
[11736]
12 4 1 0 4 0
1
[11755]
12 4 1 0 4 0
1
[11886]
12 4 1 0 4 0
1
[11914]
12 4 1 0 4 0
1
[11958]
12 4 1 0 4 0
1
[12008]
12 4 1 0 4 0
1
[12020]
12 4 1 0 4 0
1
[12085]
12 4 1 0 4 0
1
[12344]
12 4 1 0 4 0
2
[12373, 12375]
12 4 1 0 4 0
1
[12543]
12 4 1 0 4 0
1
[12631]
12 4 1 0 4 0
1
[12772]
[4, 0, 1, 6, 0, 0, [1360, 1406, 1517, 1797, 2034]]
4 0 1 6 0 0
18
[1581, 1589, 1866, 2083, 2123, 2257, 2409, 1286, 1478, 1755, 1768, 2078, 2157, 1360, 1406, 1517, 1797, 2034]
4 0 1 6 0 0
4
[1586, 1627, 2327, 2407]
4 0 1 6 0 0
2
[1745, 1975]
4 0 1 6 0 0
1
[1886]
12 4 0 0 5 0
2
[10610, 12572]
12 4 0 0 5 0
1
[10690]
12 4 0 0 5 0
1
[10705]
12 4 0 0 5 0
2
[10719, 10754]
12 4 0 0 5 0
1
[10731]
12 4 0 0 5 0
1
[10757]
12 4 0 0 5 0
1
[10880]
12 4 0 0 5 0
4
[10940, 11312, 11686, 12538]
12 4 0 0 5 0
2
[11011, 11090]
12 4 0 0 5 0
1
[11152]
12 4 0 0 5 0
1
[11160]
12 4 0 0 5 0
1
[11258]
12 4 0 0 5 0
1

12 4 1 0 4 0
1
[11273]
12 4 1 0 4 0
1
[11385]
12 4 1 0 4 0
1
[11603]
12 4 1 0 4 0
1
[11653]
12 4 1 0 4 0
1
[11681]
12 4 1 0 4 0
1
[11712]
12 4 1 0 4 0
1
[11736]
12 4 1 0 4 0
1
[11755]
12 4 1 0 4 0
1
[11886]
12 4 1 0 4 0
1
[11914]
12 4 1 0 4 0
1
[11958]
12 4 1 0 4 0
1
[12008]
12 4 1 0 4 0
1
[12020]
12 4 1 0 4 0
1
[12085]
12 4 1 0 4 0
1
[12344]
12 4 1 0 4 0
2
[12373, 12375]
12 4 1 0 4 0
1
[12543]
12 4 1 0 4 0
1
[12631]
12 4 1 0 4 0
1
[12772]
[4, 0, 1, 6, 0, 0, [1886, 1745, 1975]]
12 4 0 0 5 0
2
[10610, 12572]
12 4 0 0 5 0
1
[10690]
12 4 0 0 5 0
1
[10705]
12 4 0 0 5 0
2
[10719, 10754]
12 4 0 0 5 0
1
[10731]
12 4 0 0 5 0
1
[10757]
12 4 0 0 5 0
1
[10880]
12 4 0 0 5 0
4
[10940, 11312, 11686, 12538]
12 4 0 0 5 0
2
[11011, 11090]
12 4 0 0 5 0
1
[11152]
12 4 0 0 5 0
1
[11160]
12 4 0 0 5 0
1
[11258]
12 4 0 0 5 0
1
[11405]
12 4 0 0 5 0
1
[11464]
12 4 0 0 5 0
2
[11503, 12387]
12 4 0 0 5 0
3
[11575, 11651, 11938]
12 4 0 0 5 0
2
[11638, 12833]
12 4 0 0 5 0
1
[11735]
12 4 0 0 5 0
1
[11740]
12 4 0 0 5

[11886]
12 4 1 0 4 0
1
[11914]
12 4 1 0 4 0
1
[11958]
12 4 1 0 4 0
1
[12008]
12 4 1 0 4 0
1
[12020]
12 4 1 0 4 0
1
[12085]
12 4 1 0 4 0
1
[12344]
12 4 1 0 4 0
2
[12373, 12375]
12 4 1 0 4 0
1
[12543]
12 4 1 0 4 0
1
[12631]
12 4 1 0 4 0
1
[12772]
[12, 4, 0, 0, 5, 0, [10705]]
12 4 0 0 5 0
2
[10719, 10754]
12 4 0 0 5 0
1
[10731]
12 4 0 0 5 0
1
[10757]
12 4 0 0 5 0
1
[10880]
12 4 0 0 5 0
4
[10940, 11312, 11686, 12538]
12 4 0 0 5 0
2
[11011, 11090]
12 4 0 0 5 0
1
[11152]
12 4 0 0 5 0
1
[11160]
12 4 0 0 5 0
1
[11258]
12 4 0 0 5 0
1
[11405]
12 4 0 0 5 0
3
[11464, 10610, 12572]
12 4 0 0 5 0
2
[11503, 12387]
12 4 0 0 5 0
3
[11575, 11651, 11938]
12 4 0 0 5 0
2
[11638, 12833]
12 4 0 0 5 0
1
[11735]
12 4 0 0 5 0
1
[11740]
12 4 0 0 5 0
2
[11747, 12722]
12 4 0 0 5 0
2
[11762, 10705]
12 4 0 0 5 0
1
[11800]
12 4 0 0 5 0
1
[11949]
12 4 0 0 5 0
2
[12030, 12056]
12 4 0 0 5 0
1
[12127]
12 4 0 0 5 0
1
[12366]
12 4 0 0 5 0
1
[12506]
12 4 0 0 5 0
2
[12537, 10690]
12 4 0 0 9 0
7
[10685, 10762, 10985, 11669, 11

12 4 0 0 5 0
3
[11464, 10610, 12572]
12 4 0 0 5 0
2
[11503, 12387]
12 4 0 0 5 0
3
[11575, 11651, 11938]
12 4 0 0 5 0
2
[11638, 12833]
12 4 0 0 5 0
1
[11735]
12 4 0 0 5 0
1
[11740]
12 4 0 0 5 0
3
[11747, 12722, 10731]
12 4 0 0 5 0
2
[11762, 10705]
12 4 0 0 5 0
1
[11800]
12 4 0 0 5 0
1
[11949]
12 4 0 0 5 0
2
[12030, 12056]
12 4 0 0 5 0
1
[12127]
12 4 0 0 5 0
1
[12366]
12 4 0 0 5 0
1
[12506]
12 4 0 0 5 0
2
[12537, 10690]
12 4 0 0 9 0
7
[10685, 10762, 10985, 11669, 11693, 11743, 12584]
12 4 0 0 9 0
4
[10775, 11186, 11663, 12003]
12 4 0 0 9 0
3
[10873, 12305, 12723]
12 4 0 0 9 0
8
[10918, 11631, 11665, 11811, 11967, 12304, 12789, 12817]
12 4 0 0 10 0
4
[10674, 10766, 10886, 12594]
12 4 0 0 10 0
9
[10738, 10825, 10911, 11437, 11690, 11750, 11984, 12152, 12521]
12 4 0 0 10 0
7
[10883, 10899, 11054, 11625, 11789, 12355, 12485]
12 4 0 0 10 0
1
[11289]
12 4 1 0 3 0
1
[10621]
12 4 1 0 3 0
2
[10629, 11138]
12 4 1 0 3 0
2
[10660, 11991]
12 4 1 0 3 0
1
[10780]
12 4 1 0 3 0
2
[10783, 11806]
12 4 1 0 

...

12 4 1 0 3 0
1
[12604]
12 4 1 0 3 0
2
[12670, 10621]
12 4 1 0 3 0
1
[12738]
12 4 1 0 3 0
1
[12765]
12 4 1 0 3 0
1
[12794]
12 4 1 0 3 0
3
[12822, 11106, 10904]
12 4 1 0 3 0
2
[12841, 11725]
12 4 1 0 3 1
10
[10633, 10669, 10741, 11222, 11419, 11626, 11659, 12169, 12334, 12776]
12 4 1 0 3 1
3
[10655, 11824, 12714]
12 4 1 0 3 1
6
[10688, 11209, 11577, 11621, 12454, 12573]
12 4 1 0 3 1
8
[10810, 11441, 11456, 11639, 11845, 12167, 12294, 12399]
12 4 1 0 3 1
4
[10962, 11510, 11915, 12702]
12 4 1 0 3 1
7
[11175, 11306, 11459, 11852, 12225, 12418, 12448]
12 4 1 0 3 1
7
[11179, 11271, 11753, 11924, 12144, 12504, 12561]
12 4 1 0 3 1
1
[11976]
12 4 1 0 4 0
1
[10822]
12 4 1 0 4 0
2
[10826, 12037]
12 4 1 0 4 0
1
[10932]
12 4 1 0 4 0
1
[11038]
12 4 1 0 4 0
1
[11190]
12 4 1 0 4 0
1
[11196]
12 4 1 0 4 0
1
[11201]
12 4 1 0 4 0
1
[11204]
12 4 1 0 4 0
1
[11273]
12 4 1 0 4 0
1
[11385]
12 4 1 0 4 0
1
[11603]
12 4 1 0 4 0
1
[11653]
12 4 1 0 4 0
1
[11681]
12 4 1 0 4 0
1
[11712]
12 4 1 0 4 0
1
[11736]
12 4 1 0

12 4 1 0 4 0
1
[11712]
12 4 1 0 4 0
1
[11736]
12 4 1 0 4 0
1
[11755]
12 4 1 0 4 0
1
[11886]
12 4 1 0 4 0
1
[11914]
12 4 1 0 4 0
1
[11958]
12 4 1 0 4 0
1
[12008]
12 4 1 0 4 0
1
[12020]
12 4 1 0 4 0
1
[12085]
12 4 1 0 4 0
1
[12344]
12 4 1 0 4 0
2
[12373, 12375]
12 4 1 0 4 0
1
[12543]
12 4 1 0 4 0
1
[12631]
12 4 1 0 4 0
1
[12772]
[12, 4, 1, 0, 3, 0, [12202]]
12 4 1 0 3 0
1
[12203]
12 4 1 0 3 0
6
[12206, 10783, 11806, 11128, 11737, 12323]
12 4 1 0 3 0
2
[12210, 10984]
12 4 1 0 3 0
8
[12255, 11393, 10660, 11991, 11540, 11903, 11629, 12741]
12 4 1 0 3 0
2
[12260, 11769]
12 4 1 0 3 0
1
[12326]
12 4 1 0 3 0
2
[12331, 11908]
12 4 1 0 3 0
3
[12338, 11116, 11109]
12 4 1 0 3 0
1
[12359]
12 4 1 0 3 0
1
[12444]
12 4 1 0 3 0
3
[12530, 11723, 12202]
12 4 1 0 3 0
1
[12593]
12 4 1 0 3 0
1
[12604]
12 4 1 0 3 0
12
[12670, 10621, 11955, 11297, 11779, 11827, 11909, 10964, 10629, 11138, 11848, 11217]
12 4 1 0 3 0
1
[12738]
12 4 1 0 3 0
4
[12765, 12195, 12034, 11927]
12 4 1 0 3 0
1
[12794]
12 4 1 0 3 0
8
[128

12 4 1 0 3 0
13
[12530, 11723, 12202, 12331, 11908, 12255, 11393, 10660, 11991, 11540, 11903, 11629, 12741]
12 4 1 0 3 0
1
[12593]
12 4 1 0 3 0
1
[12604]
12 4 1 0 3 0
12
[12670, 10621, 11955, 11297, 11779, 11827, 11909, 10964, 10629, 11138, 11848, 11217]
12 4 1 0 3 0
2
[12738, 12203]
12 4 1 0 3 0
4
[12765, 12195, 12034, 11927]
12 4 1 0 3 0
3
[12794, 12210, 10984]
12 4 1 0 3 0
16
[12822, 11106, 10904, 12155, 10780, 10853, 11307, 11275, 12206, 10783, 11806, 11128, 11737, 12323, 12260, 11769]
12 4 1 0 3 0
3
[12841, 11725, 12326]
12 4 1 0 3 1
10
[10633, 10669, 10741, 11222, 11419, 11626, 11659, 12169, 12334, 12776]
12 4 1 0 3 1
3
[10655, 11824, 12714]
12 4 1 0 3 1
6
[10688, 11209, 11577, 11621, 12454, 12573]
12 4 1 0 3 1
8
[10810, 11441, 11456, 11639, 11845, 12167, 12294, 12399]
12 4 1 0 3 1
4
[10962, 11510, 11915, 12702]
12 4 1 0 3 1
7
[11175, 11306, 11459, 11852, 12225, 12418, 12448]
12 4 1 0 3 1
7
[11179, 11271, 11753, 11924, 12144, 12504, 12561]
12 4 1 0 3 1
1
[11976]
12 4 1 0 4 0
1
[1

[11958]
12 4 1 0 4 0
1
[12008]
12 4 1 0 4 0
1
[12020]
12 4 1 0 4 0
1
[12085]
12 4 1 0 4 0
1
[12344]
12 4 1 0 4 0
2
[12373, 12375]
12 4 1 0 4 0
1
[12543]
12 4 1 0 4 0
1
[12631]
12 4 1 0 4 0
1
[12772]
[12, 4, 1, 0, 3, 0, [12765, 12195, 12034, 11927, 12530, 11723, 12202, 12331, 11908, 12255, 11393, 10660, 11991, 11540, 11903, 11629, 12741]]
12 4 1 0 3 0
3
[12841, 11725, 12326]
12 4 1 0 3 1
10
[10633, 10669, 10741, 11222, 11419, 11626, 11659, 12169, 12334, 12776]
12 4 1 0 3 1
3
[10655, 11824, 12714]
12 4 1 0 3 1
6
[10688, 11209, 11577, 11621, 12454, 12573]
12 4 1 0 3 1
8
[10810, 11441, 11456, 11639, 11845, 12167, 12294, 12399]
12 4 1 0 3 1
4
[10962, 11510, 11915, 12702]
12 4 1 0 3 1
7
[11175, 11306, 11459, 11852, 12225, 12418, 12448]
12 4 1 0 3 1
7
[11179, 11271, 11753, 11924, 12144, 12504, 12561]
12 4 1 0 3 1
1
[11976]
12 4 1 0 4 0
1
[10822]
12 4 1 0 4 0
2
[10826, 12037]
12 4 1 0 4 0
1
[10932]
12 4 1 0 4 0
1
[11038]
12 4 1 0 4 0
1
[11190]
12 4 1 0 4 0
1
[11196]
12 4 1 0 4 0
1
[11201]
12 4

12 4 1 0 4 0
2
[11603, 11038]
12 4 1 0 4 0
1
[11653]
12 4 1 0 4 0
1
[11681]
12 4 1 0 4 0
1
[11712]
12 4 1 0 4 0
1
[11736]
12 4 1 0 4 0
1
[11755]
12 4 1 0 4 0
1
[11886]
12 4 1 0 4 0
1
[11914]
12 4 1 0 4 0
3
[11958, 10826, 12037]
12 4 1 0 4 0
1
[12008]
12 4 1 0 4 0
1
[12020]
12 4 1 0 4 0
1
[12085]
12 4 1 0 4 0
1
[12344]
12 4 1 0 4 0
2
[12373, 12375]
12 4 1 0 4 0
1
[12543]
12 4 1 0 4 0
3
[12631, 10932, 10822]
12 4 1 0 4 0
1
[12772]
[12, 4, 1, 0, 4, 0, [11190]]
12 4 1 0 4 0
2
[11196, 11190]
12 4 1 0 4 0
1
[11201]
12 4 1 0 4 0
1
[11204]
12 4 1 0 4 0
1
[11273]
12 4 1 0 4 0
1
[11385]
12 4 1 0 4 0
2
[11603, 11038]
12 4 1 0 4 0
1
[11653]
12 4 1 0 4 0
1
[11681]
12 4 1 0 4 0
1
[11712]
12 4 1 0 4 0
1
[11736]
12 4 1 0 4 0
1
[11755]
12 4 1 0 4 0
1
[11886]
12 4 1 0 4 0
1
[11914]
12 4 1 0 4 0
3
[11958, 10826, 12037]
12 4 1 0 4 0
1
[12008]
12 4 1 0 4 0
1
[12020]
12 4 1 0 4 0
1
[12085]
12 4 1 0 4 0
1
[12344]
12 4 1 0 4 0
2
[12373, 12375]
12 4 1 0 4 0
1
[12543]
12 4 1 0 4 0
3
[12631, 10932, 10822]
12 4 1

In [864]:
start = time.time()
data_address_company_anonymized_5 = copy.deepcopy(data_address_company_anonymized_df_4.values)
column = list(data_address_company_anonymized_df_4.columns)
tmp = data_address_company_anonymized_df_4["address_float_6"].values
for bb in group_6_backup:
    for aa in bb:
        for v in aa:
            for p in v:
                for q in p:
                    for y in q:
                        for i in y:
                            a = 0
                            anonymized_data = []
                            for x,j in enumerate(i):
                                if x == 0:
                                    fix = tmp[j]
                                elif fix != tmp[j]:
                                    a+=1
                                    for k in range(1,64):
                                        if fix[0:-k] == tmp[j][0:-k]:
                                            anonymized_data.append(fix[0:-k] + "".join("0" for l in range(k)))
                                            break
                            if a > 0:
                                for x,anony in enumerate(anonymized_data):
                                    if x == 0:
                                        fix = anony
                                    elif fix != anony:
                                        for k in range(1,64):
                                            if fix[0:-k] == anony[0:-k]:
                                                fix = (fix[0:-k] + "".join("0" for l in range(k)))
                                                break
                                column_num = column.index("address_float_6")
                                for m in i:
                                    data_address_company_anonymized_5[m,column_num] = fix
                                    for o in range(column_num+1,len(column)):
                                        if o%2 == 1:
                                            data_address_company_anonymized_5[m,o] = "".join("0" for l in range(64))
                                        else:
                                            data_address_company_anonymized_5[m,o] = np.float64(0)
print(time.time()-start)

1.007824182510376


In [865]:
data_address_company_anonymized_df_5 = pd.DataFrame(data_address_company_anonymized_5,columns=column)

In [866]:
group_7 = []
tmp = list(data_address_company_anonymized_df_5["address_float_7"].unique())
tmp_1 = list(data_address_company_anonymized_df_5.address_float_7.values)
start = time.time()
for bb in group_6_backup:
    roupgroup_7 = []
    for aa in bb:
        uopgroup_7 = []
        for q in aa:
            start = time.time()
            opgroup_7 = []
            for n in q:
                pgroup_7 = []
                for m in n:
                    grou_7 = []
                    for i in m:
                        gro_7 = []
                        for x,j in enumerate(i):
                            gr_7 = []
                            for l in tmp:
                                g_7 = []
                                for k in j:
                                    if tmp_1[k] == l:
                                        g_7.append(k)
                                if len(g_7) != 0:
                                    gr_7.append(g_7)
                            #print(gro_7)
                            gro_7.append(gr_7)
                        grou_7.append(gro_7)
                    pgroup_7.append(grou_7)
                opgroup_7.append(pgroup_7)
            uopgroup_7.append(opgroup_7)
        roupgroup_7.append(uopgroup_7)
    group_7.append(roupgroup_7)
print(time.time()-start)

0.0007941722869873047


In [867]:
group_7_backup = copy.deepcopy(group_7)

In [868]:
group_7_not_anonymized = []
for xx,mm in enumerate(group_7):
    for yy,ii in enumerate(mm):
        for zz,jj in enumerate(ii):
            for p,n in enumerate(jj):
                for x,m in enumerate(n):
                    for y,i in enumerate(m):
                            for z,j in enumerate(i):
                                for k in j:
                                    if 0<len(k)<20:
                                        print(xx,yy,zz,p,x,y,z)
                                        print(len(k))
                                        print(k)
                                        group_7_not_anonymized.append([xx,yy,zz,p,x,y,z,k])

12 4 0 0 9 0 0
1
[10665]
12 4 0 0 9 0 0
1
[10715]
12 4 0 0 9 0 0
1
[10721]
12 4 0 0 9 0 0
2
[11079, 12161]
12 4 0 0 9 0 0
3
[11356, 11402, 11729]
12 4 0 0 9 0 0
1
[11370]
12 4 0 0 9 0 0
1
[11432]
12 4 0 0 9 0 0
1
[11518]
12 4 0 0 9 0 0
1
[11813]
12 4 0 0 9 0 0
1
[11832]
12 4 0 0 9 0 0
1
[11834]
12 4 0 0 9 0 0
1
[11880]
12 4 0 0 9 0 0
1
[11893]
12 4 0 0 9 0 0
1
[12075]
12 4 0 0 9 0 0
1
[12099]
12 4 0 0 9 0 0
1
[12100]
12 4 0 0 9 0 0
1
[12188]
12 4 0 0 9 0 0
1
[12239]
12 4 0 0 9 0 0
1
[12477]
12 4 1 0 0 0 0
1
[10595]
12 4 1 0 0 0 0
1
[12663]
12 4 1 0 0 0 0
1
[10670]
12 4 1 0 0 0 0
1
[10733]
12 4 1 0 0 0 0
2
[10759, 10839]
12 4 1 0 0 0 0
1
[10835]
12 4 1 0 0 0 0
1
[12312]
12 4 1 0 0 0 0
2
[11724, 12493]
12 4 1 0 0 0 0
1
[11268]
12 4 1 0 0 0 0
2
[11310, 12306]
12 4 1 0 0 0 0
2
[11322, 11923]
12 4 1 0 0 0 0
1
[11327]
12 4 1 0 0 0 0
3
[11328, 11857, 12595]
12 4 1 0 0 0 0
2
[11377, 12810]
12 4 1 0 0 0 0
1
[11531]
12 4 1 0 0 0 0
1
[11561]
12 4 1 0 0 0 0
3
[12376, 12377, 12384]
12 4 1 0 0 0 0
2

In [869]:
tmp = data_address_company_anonymized_df_5.address_float_value_7
start_1 = time.time()
while len(group_7_not_anonymized) != 0:
    a = 0
    start = time.time()
#for i in group_7_not_anonymized:
    for x,j in enumerate(group_7_backup[group_7_not_anonymized[0][0]][group_7_not_anonymized[0][1]][group_7_not_anonymized[0][2]][group_7_not_anonymized[0][3]][group_7_not_anonymized[0][4]][group_7_not_anonymized[0][5]][group_7_not_anonymized[0][6]]):
        if len(j) != 0:
            if j[0] != group_7_not_anonymized[0][7][0]:
                if a == 0:
                    diff = abs(tmp[j[0]]-tmp[group_7_not_anonymized[0][7][0]])
                    closest_group = x
                    a += 1
                elif diff > abs(tmp[j[0]]-tmp[group_7_not_anonymized[0][7][0]]):
                    ##print(diff)
                    ##print(tmp[j[0]]-tmp[group_7_not_anonymized[0][1][0]])
                    diff = abs(tmp[j[0]]-tmp[group_7_not_anonymized[0][7][0]])
                    closest_group = x
    group_7_backup[group_7_not_anonymized[0][0]][group_7_not_anonymized[0][1]][group_7_not_anonymized[0][2]][group_7_not_anonymized[0][3]][group_7_not_anonymized[0][4]][group_7_not_anonymized[0][5]][group_7_not_anonymized[0][6]][closest_group].extend(group_7_not_anonymized[0][7])
    group_7_backup[group_7_not_anonymized[0][0]][group_7_not_anonymized[0][1]][group_7_not_anonymized[0][2]][group_7_not_anonymized[0][3]][group_7_not_anonymized[0][4]][group_7_not_anonymized[0][5]][group_7_not_anonymized[0][6]].remove(group_7_not_anonymized[0][7])
    group_7_not_anonymized = []
    for xx,mm in enumerate(group_7_backup):
        for yy,ii in enumerate(mm):
            for zz,jj in enumerate(ii):
                for p,n in enumerate(jj):
                    for x,m in enumerate(n):
                        for y,i in enumerate(m):
                                for z,j in enumerate(i):
                                    for k in j:
                                        if 0<len(k)<20:
                                            print(xx,yy,zz,p,x,y,z)
                                            print(len(k))
                                            print(k)
                                            group_7_not_anonymized.append([xx,yy,zz,p,x,y,z,k])
    if len(group_7_not_anonymized) != 0:
        print(group_7_not_anonymized[0])
    #print(time.time()-start)
print(time.time()-start_1)

12 4 0 0 9 0 0
1
[10715]
12 4 0 0 9 0 0
1
[10721]
12 4 0 0 9 0 0
2
[11079, 12161]
12 4 0 0 9 0 0
3
[11356, 11402, 11729]
12 4 0 0 9 0 0
1
[11370]
12 4 0 0 9 0 0
1
[11432]
12 4 0 0 9 0 0
1
[11518]
12 4 0 0 9 0 0
1
[11813]
12 4 0 0 9 0 0
1
[11832]
12 4 0 0 9 0 0
1
[11834]
12 4 0 0 9 0 0
1
[11880]
12 4 0 0 9 0 0
1
[11893]
12 4 0 0 9 0 0
1
[12075]
12 4 0 0 9 0 0
1
[12099]
12 4 0 0 9 0 0
1
[12100]
12 4 0 0 9 0 0
1
[12188]
12 4 0 0 9 0 0
2
[12239, 10665]
12 4 0 0 9 0 0
1
[12477]
12 4 1 0 0 0 0
1
[10595]
12 4 1 0 0 0 0
1
[12663]
12 4 1 0 0 0 0
1
[10670]
12 4 1 0 0 0 0
1
[10733]
12 4 1 0 0 0 0
2
[10759, 10839]
12 4 1 0 0 0 0
1
[10835]
12 4 1 0 0 0 0
1
[12312]
12 4 1 0 0 0 0
2
[11724, 12493]
12 4 1 0 0 0 0
1
[11268]
12 4 1 0 0 0 0
2
[11310, 12306]
12 4 1 0 0 0 0
2
[11322, 11923]
12 4 1 0 0 0 0
1
[11327]
12 4 1 0 0 0 0
3
[11328, 11857, 12595]
12 4 1 0 0 0 0
2
[11377, 12810]
12 4 1 0 0 0 0
1
[11531]
12 4 1 0 0 0 0
1
[11561]
12 4 1 0 0 0 0
3
[12376, 12377, 12384]
12 4 1 0 0 0 0
2
[12599, 12600]
12

12 4 1 0 0 2 0
3
[10829, 11627, 12117]
12 4 1 0 0 2 0
4
[10844, 11476, 11960, 12551]
12 4 1 0 0 2 0
2
[10862, 11015]
12 4 1 0 0 2 0
3
[10882, 11765, 12614]
12 4 1 0 0 2 0
4
[10892, 11345, 11786, 12287]
12 4 1 0 0 2 0
1
[10957]
12 4 1 0 0 2 0
1
[11088]
12 4 1 0 0 2 0
1
[11107]
12 4 1 0 0 2 0
1
[11124]
12 4 1 0 0 2 0
1
[11185]
12 4 1 0 0 2 0
1
[11346]
12 4 1 0 0 2 0
2
[11308, 11721]
12 4 1 0 0 2 0
1
[11309]
12 4 1 0 0 2 0
1
[12651]
12 4 1 0 0 2 0
4
[11364, 11389, 11448, 11796]
12 4 1 0 0 2 0
1
[11396]
12 4 1 0 0 2 0
1
[11435]
12 4 1 0 0 2 0
1
[11449]
12 4 1 0 0 2 0
1
[11473]
12 4 1 0 0 2 0
3
[11541, 11689, 12582]
12 4 1 0 0 2 0
3
[11617, 11754, 12367]
12 4 1 0 0 2 0
1
[11670]
12 4 1 0 0 2 0
1
[11757]
12 4 1 0 0 2 0
1
[11828]
12 4 1 0 0 2 0
1
[11928]
12 4 1 0 0 2 0
1
[11937]
12 4 1 0 0 2 0
1
[12295]
12 4 1 0 0 2 0
1
[12335]
12 4 1 0 0 2 0
1
[12491]
12 4 1 0 0 2 0
1
[12494]
12 4 1 0 0 2 0
1
[12588]
12 4 1 0 0 2 0
1
[12666]
[12, 4, 0, 0, 9, 0, 0, [11356, 11402, 11729]]
12 4 0 0 9 0 0
1
[113

12 4 0 0 9 0 0
1
[12100]
12 4 0 0 9 0 0
1
[12188]
12 4 0 0 9 0 0
4
[12239, 10665, 11079, 12161]
12 4 0 0 9 0 0
1
[12477]
12 4 1 0 0 0 0
1
[10595]
12 4 1 0 0 0 0
1
[12663]
12 4 1 0 0 0 0
1
[10670]
12 4 1 0 0 0 0
1
[10733]
12 4 1 0 0 0 0
2
[10759, 10839]
12 4 1 0 0 0 0
1
[10835]
12 4 1 0 0 0 0
1
[12312]
12 4 1 0 0 0 0
2
[11724, 12493]
12 4 1 0 0 0 0
1
[11268]
12 4 1 0 0 0 0
2
[11310, 12306]
12 4 1 0 0 0 0
2
[11322, 11923]
12 4 1 0 0 0 0
1
[11327]
12 4 1 0 0 0 0
3
[11328, 11857, 12595]
12 4 1 0 0 0 0
2
[11377, 12810]
12 4 1 0 0 0 0
1
[11531]
12 4 1 0 0 0 0
1
[11561]
12 4 1 0 0 0 0
3
[12376, 12377, 12384]
12 4 1 0 0 0 0
2
[12599, 12600]
12 4 1 0 0 1 0
1
[10596]
12 4 1 0 0 1 0
1
[10602]
12 4 1 0 0 1 0
2
[10634, 12419]
12 4 1 0 0 1 0
2
[10747, 11781]
12 4 1 0 0 1 0
2
[12417, 12709]
12 4 1 0 0 1 0
1
[11478]
12 4 1 0 0 1 0
1
[11590]
12 4 1 0 0 1 0
1
[11552]
12 4 1 0 0 1 0
2
[11010, 11255]
12 4 1 0 0 1 0
1
[11060]
12 4 1 0 0 1 0
2
[11094, 11320]
12 4 1 0 0 1 0
1
[11110]
12 4 1 0 0 1 0
1
[11164]

[12138]
12 4 1 0 0 1 0
1
[12526]
12 4 1 0 0 2 0
1
[12215]
12 4 1 0 0 2 0
3
[11121, 11170, 12727]
12 4 1 0 0 2 0
1
[10703]
12 4 1 0 0 2 0
1
[10739]
12 4 1 0 0 2 0
6
[10751, 11162, 12397, 12519, 12628, 12798]
12 4 1 0 0 2 0
3
[10829, 11627, 12117]
12 4 1 0 0 2 0
4
[10844, 11476, 11960, 12551]
12 4 1 0 0 2 0
2
[10862, 11015]
12 4 1 0 0 2 0
3
[10882, 11765, 12614]
12 4 1 0 0 2 0
4
[10892, 11345, 11786, 12287]
12 4 1 0 0 2 0
1
[10957]
12 4 1 0 0 2 0
1
[11088]
12 4 1 0 0 2 0
1
[11107]
12 4 1 0 0 2 0
1
[11124]
12 4 1 0 0 2 0
1
[11185]
12 4 1 0 0 2 0
1
[11346]
12 4 1 0 0 2 0
2
[11308, 11721]
12 4 1 0 0 2 0
1
[11309]
12 4 1 0 0 2 0
1
[12651]
12 4 1 0 0 2 0
4
[11364, 11389, 11448, 11796]
12 4 1 0 0 2 0
1
[11396]
12 4 1 0 0 2 0
1
[11435]
12 4 1 0 0 2 0
1
[11449]
12 4 1 0 0 2 0
1
[11473]
12 4 1 0 0 2 0
3
[11541, 11689, 12582]
12 4 1 0 0 2 0
3
[11617, 11754, 12367]
12 4 1 0 0 2 0
1
[11670]
12 4 1 0 0 2 0
1
[11757]
12 4 1 0 0 2 0
1
[11828]
12 4 1 0 0 2 0
1
[11928]
12 4 1 0 0 2 0
1
[11937]
12 4 1 0 0

12 4 1 0 0 0 0
2
[12599, 12600]
12 4 1 0 0 1 0
1
[10596]
12 4 1 0 0 1 0
1
[10602]
12 4 1 0 0 1 0
2
[10634, 12419]
12 4 1 0 0 1 0
2
[10747, 11781]
12 4 1 0 0 1 0
2
[12417, 12709]
12 4 1 0 0 1 0
1
[11478]
12 4 1 0 0 1 0
1
[11590]
12 4 1 0 0 1 0
1
[11552]
12 4 1 0 0 1 0
2
[11010, 11255]
12 4 1 0 0 1 0
1
[11060]
12 4 1 0 0 1 0
2
[11094, 11320]
12 4 1 0 0 1 0
1
[11110]
12 4 1 0 0 1 0
1
[11164]
12 4 1 0 0 1 0
1
[11202]
12 4 1 0 0 1 0
1
[11210]
12 4 1 0 0 1 0
1
[11235]
12 4 1 0 0 1 0
1
[11252]
12 4 1 0 0 1 0
2
[11772, 12319]
12 4 1 0 0 1 0
2
[11314, 12637]
12 4 1 0 0 1 0
1
[11348]
12 4 1 0 0 1 0
3
[11368, 11775, 12428]
12 4 1 0 0 1 0
1
[11398]
12 4 1 0 0 1 0
1
[11582]
12 4 1 0 0 1 0
1
[11709]
12 4 1 0 0 1 0
1
[11787]
12 4 1 0 0 1 0
1
[11856]
12 4 1 0 0 1 0
1
[12040]
12 4 1 0 0 1 0
1
[12110]
12 4 1 0 0 1 0
1
[12138]
12 4 1 0 0 1 0
1
[12526]
12 4 1 0 0 2 0
1
[12215]
12 4 1 0 0 2 0
3
[11121, 11170, 12727]
12 4 1 0 0 2 0
1
[10703]
12 4 1 0 0 2 0
1
[10739]
12 4 1 0 0 2 0
6
[10751, 11162, 12397, 12

...

12 4 1 0 0 1 0
4
[11348, 10596, 11478, 11164]
12 4 1 0 0 1 0
7
[11368, 11775, 12428, 10602, 11235, 10634, 12419]
12 4 1 0 0 1 0
6
[11398, 10747, 11781, 11110, 11552, 11210]
12 4 1 0 0 1 0
3
[11582, 11772, 12319]
12 4 1 0 0 1 0
1
[11709]
12 4 1 0 0 1 0
2
[11787, 11202]
12 4 1 0 0 1 0
3
[11856, 11094, 11320]
12 4 1 0 0 1 0
1
[12040]
12 4 1 0 0 1 0
3
[12110, 12417, 12709]
12 4 1 0 0 1 0
4
[12138, 11010, 11255, 11252]
12 4 1 0 0 1 0
2
[12526, 11590]
12 4 1 0 0 2 0
1
[12215]
12 4 1 0 0 2 0
3
[11121, 11170, 12727]
12 4 1 0 0 2 0
1
[10703]
12 4 1 0 0 2 0
1
[10739]
12 4 1 0 0 2 0
6
[10751, 11162, 12397, 12519, 12628, 12798]
12 4 1 0 0 2 0
3
[10829, 11627, 12117]
12 4 1 0 0 2 0
4
[10844, 11476, 11960, 12551]
12 4 1 0 0 2 0
2
[10862, 11015]
12 4 1 0 0 2 0
3
[10882, 11765, 12614]
12 4 1 0 0 2 0
4
[10892, 11345, 11786, 12287]
12 4 1 0 0 2 0
1
[10957]
12 4 1 0 0 2 0
1
[11088]
12 4 1 0 0 2 0
1
[11107]
12 4 1 0 0 2 0
1
[11124]
12 4 1 0 0 2 0
1
[11185]
12 4 1 0 0 2 0
1
[11346]
12 4 1 0 0 2 0
2
[11308,

12 4 1 0 0 2 0
2
[10862, 11015]
12 4 1 0 0 2 0
3
[10882, 11765, 12614]
12 4 1 0 0 2 0
4
[10892, 11345, 11786, 12287]
12 4 1 0 0 2 0
1
[10957]
12 4 1 0 0 2 0
1
[11088]
12 4 1 0 0 2 0
1
[11107]
12 4 1 0 0 2 0
1
[11124]
12 4 1 0 0 2 0
1
[11185]
12 4 1 0 0 2 0
1
[11346]
12 4 1 0 0 2 0
2
[11308, 11721]
12 4 1 0 0 2 0
1
[11309]
12 4 1 0 0 2 0
1
[12651]
12 4 1 0 0 2 0
4
[11364, 11389, 11448, 11796]
12 4 1 0 0 2 0
1
[11396]
12 4 1 0 0 2 0
1
[11435]
12 4 1 0 0 2 0
1
[11449]
12 4 1 0 0 2 0
1
[11473]
12 4 1 0 0 2 0
3
[11541, 11689, 12582]
12 4 1 0 0 2 0
3
[11617, 11754, 12367]
12 4 1 0 0 2 0
1
[11670]
12 4 1 0 0 2 0
1
[11757]
12 4 1 0 0 2 0
1
[11828]
12 4 1 0 0 2 0
1
[11928]
12 4 1 0 0 2 0
1
[11937]
12 4 1 0 0 2 0
1
[12295]
12 4 1 0 0 2 0
1
[12335]
12 4 1 0 0 2 0
1
[12491]
12 4 1 0 0 2 0
1
[12494]
12 4 1 0 0 2 0
1
[12588]
12 4 1 0 0 2 0
1
[12666]
[12, 4, 1, 0, 0, 1, 0, [11856, 11094, 11320, 11787, 11202, 11314, 12637, 11060]]
12 4 1 0 0 1 0
14
[12040, 11398, 10747, 11781, 11110, 11552, 11210, 115

12 4 1 0 0 2 0
1
[11124]
12 4 1 0 0 2 0
1
[11185]
12 4 1 0 0 2 0
1
[11346]
12 4 1 0 0 2 0
2
[11308, 11721]
12 4 1 0 0 2 0
1
[11309]
12 4 1 0 0 2 0
1
[12651]
12 4 1 0 0 2 0
4
[11364, 11389, 11448, 11796]
12 4 1 0 0 2 0
2
[11396, 10739]
12 4 1 0 0 2 0
1
[11435]
12 4 1 0 0 2 0
1
[11449]
12 4 1 0 0 2 0
1
[11473]
12 4 1 0 0 2 0
3
[11541, 11689, 12582]
12 4 1 0 0 2 0
3
[11617, 11754, 12367]
12 4 1 0 0 2 0
1
[11670]
12 4 1 0 0 2 0
1
[11757]
12 4 1 0 0 2 0
1
[11828]
12 4 1 0 0 2 0
1
[11928]
12 4 1 0 0 2 0
4
[11937, 11121, 11170, 12727]
12 4 1 0 0 2 0
1
[12295]
12 4 1 0 0 2 0
2
[12335, 12215]
12 4 1 0 0 2 0
8
[12491, 10703, 10751, 11162, 12397, 12519, 12628, 12798]
12 4 1 0 0 2 0
1
[12494]
12 4 1 0 0 2 0
1
[12588]
12 4 1 0 0 2 0
1
[12666]
[12, 4, 1, 0, 0, 2, 0, [10829, 11627, 12117]]
12 4 1 0 0 2 0
4
[10844, 11476, 11960, 12551]
12 4 1 0 0 2 0
2
[10862, 11015]
12 4 1 0 0 2 0
3
[10882, 11765, 12614]
12 4 1 0 0 2 0
4
[10892, 11345, 11786, 12287]
12 4 1 0 0 2 0
1
[10957]
12 4 1 0 0 2 0
1
[11088]
1

12 4 1 0 0 2 0
3
[11396, 10739, 11309]
12 4 1 0 0 2 0
8
[11435, 11308, 11721, 10892, 11345, 11786, 12287, 11346]
12 4 1 0 0 2 0
1
[11449]
12 4 1 0 0 2 0
2
[11473, 11088]
12 4 1 0 0 2 0
3
[11541, 11689, 12582]
12 4 1 0 0 2 0
3
[11617, 11754, 12367]
12 4 1 0 0 2 0
1
[11670]
12 4 1 0 0 2 0
3
[11757, 10862, 11015]
12 4 1 0 0 2 0
2
[11828, 11124]
12 4 1 0 0 2 0
1
[11928]
12 4 1 0 0 2 0
4
[11937, 11121, 11170, 12727]
12 4 1 0 0 2 0
4
[12295, 10829, 11627, 12117]
12 4 1 0 0 2 0
2
[12335, 12215]
12 4 1 0 0 2 0
8
[12491, 10703, 10751, 11162, 12397, 12519, 12628, 12798]
12 4 1 0 0 2 0
1
[12494]
12 4 1 0 0 2 0
2
[12588, 11107]
12 4 1 0 0 2 0
5
[12666, 10882, 11765, 12614, 11185]
[12, 4, 1, 0, 0, 2, 0, [12651, 10844, 11476, 11960, 12551]]
12 4 1 0 0 2 0
5
[11364, 11389, 11448, 11796, 10957]
12 4 1 0 0 2 0
3
[11396, 10739, 11309]
12 4 1 0 0 2 0
8
[11435, 11308, 11721, 10892, 11345, 11786, 12287, 11346]
12 4 1 0 0 2 0
1
[11449]
12 4 1 0 0 2 0
2
[11473, 11088]
12 4 1 0 0 2 0
3
[11541, 11689, 12582]
1

12 4 1 0 0 2 0
5
[12666, 10882, 11765, 12614, 11185]
[12, 4, 1, 0, 0, 2, 0, [12335, 12215, 11928]]
12 4 1 0 0 2 0
16
[12491, 10703, 10751, 11162, 12397, 12519, 12628, 12798, 11449, 11828, 11124, 11473, 11088, 11757, 10862, 11015]
12 4 1 0 0 2 0
4
[12494, 12335, 12215, 11928]
12 4 1 0 0 2 0
9
[12588, 11107, 11937, 11121, 11170, 12727, 11617, 11754, 12367]
12 4 1 0 0 2 0
5
[12666, 10882, 11765, 12614, 11185]
[12, 4, 1, 0, 0, 2, 0, [12491, 10703, 10751, 11162, 12397, 12519, 12628, 12798, 11449, 11828, 11124, 11473, 11088, 11757, 10862, 11015]]
12 4 1 0 0 2 0
4
[12494, 12335, 12215, 11928]
12 4 1 0 0 2 0
5
[12666, 10882, 11765, 12614, 11185]
[12, 4, 1, 0, 0, 2, 0, [12494, 12335, 12215, 11928]]
12 4 1 0 0 2 0
5
[12666, 10882, 11765, 12614, 11185]
[12, 4, 1, 0, 0, 2, 0, [12666, 10882, 11765, 12614, 11185]]
8.25203251838684


In [870]:
start = time.time()
data_address_company_anonymized_6 = copy.deepcopy(data_address_company_anonymized_df_5.values)
column = list(data_address_company_anonymized_df_5.columns)
tmp = data_address_company_anonymized_df_5["address_float_7"].values
for cc in group_7_backup:
    for bb in cc:
        for aa in bb:
            for v in aa:
                for p in v:
                    for q in p:
                        for y in q:
                            for i in y:
                                a = 0
                                anonymized_data = []
                                for x,j in enumerate(i):
                                    if x == 0:
                                        fix = tmp[j]
                                    elif fix != tmp[j]:
                                        a+=1
                                        for k in range(1,64):
                                            if fix[0:-k] == tmp[j][0:-k]:
                                                anonymized_data.append(fix[0:-k] + "".join("0" for l in range(k)))
                                                break
                                if a > 0:
                                    for x,anony in enumerate(anonymized_data):
                                        if x == 0:
                                            fix = anony
                                        elif fix != anony:
                                            for k in range(1,64):
                                                if fix[0:-k] == anony[0:-k]:
                                                    fix = (fix[0:-k] + "".join("0" for l in range(k)))
                                                    break
                                    column_num = column.index("address_float_7")
                                    for m in i:
                                        data_address_company_anonymized_6[m,column_num] = fix
                                        for o in range(column_num+1,len(column)):
                                            if o%2 == 1:
                                                data_address_company_anonymized_6[m,o] = "".join("0" for l in range(64))
                                            else:
                                                data_address_company_anonymized_6[m,o] = np.float64(0)
print(time.time()-start)

1.0556459426879883


In [871]:
data_address_company_anonymized_df_6 = pd.DataFrame(data_address_company_anonymized_6,columns=column)

In [872]:
group_8 = []
tmp = list(data_address_company_anonymized_df_6["address_float_8"].unique())
tmp_1 = list(data_address_company_anonymized_df_6.address_float_8.values)
start = time.time()
for cc in group_7_backup:
    groupgroup_8 = []
    for bb in group_6_backup:
        roupgroup_8 = []
        for aa in bb:
            uopgroup_8 = []
            for q in aa:
                start = time.time()
                opgroup_8 = []
                for n in q:
                    pgroup_8 = []
                    for m in n:
                        grou_8 = []
                        for i in m:
                            gro_8 = []
                            for x,j in enumerate(i):
                                gr_8 = []
                                for l in tmp:
                                    g_8 = []
                                    for k in j:
                                        if tmp_1[k] == l:
                                            g_8.append(k)
                                    if len(g_8) != 0:
                                        gr_8.append(g_8)
                                #print(gro_8)
                                gro_8.append(gr_8)
                            grou_8.append(gro_8)
                        pgroup_8.append(grou_8)
                    opgroup_8.append(pgroup_8)
                uopgroup_8.append(opgroup_8)
            roupgroup_8.append(uopgroup_8)
        groupgroup_8.append(roupgroup_8)
    group_8.append(groupgroup_8)
print(time.time()-start)

0.00016355514526367188


In [873]:
group_8_backup = copy.deepcopy(group_8)

In [874]:
group_8_not_anonymized = []
for pp,nn in enumerate(group_8):
    for xx,mm in enumerate(nn):
        for yy,ii in enumerate(mm):
            for zz,jj in enumerate(ii):
                for p,n in enumerate(jj):
                    for x,m in enumerate(n):
                        for y,i in enumerate(m):
                                for z,j in enumerate(i):
                                    for k in j:
                                        if 0<len(k)<20:
                                            print(pp,xx,yy,zz,p,x,y,z)
                                            print(len(k))
                                            print(k)
                                            group_8_not_anonymized.append([pp,xx,yy,zz,p,x,y,z,k])

In [875]:
tmp = data_address_company_anonymized_df_6.address_float_value_8
start_1 = time.time()
while len(group_8_not_anonymized) != 0:
    a = 0
    start = time.time()
#for i in group_8_not_anonymized:
    for x,j in enumerate(group_8_backup[group_8_not_anonymized[0][0]][group_8_not_anonymized[0][1]][group_8_not_anonymized[0][2]][group_8_not_anonymized[0][3]][group_8_not_anonymized[0][4]][group_8_not_anonymized[0][5]][group_8_not_anonymized[0][6]][group_8_not_anonymized[0][7]]):
        if len(j) != 0:
            if j[0] != group_8_not_anonymized[0][8][0]:
                if a == 0:
                    diff = abs(tmp[j[0]]-tmp[group_8_not_anonymized[0][8][0]])
                    closest_group = x
                    a += 1
                elif diff > abs(tmp[j[0]]-tmp[group_8_not_anonymized[0][8][0]]):
                    ##print(diff)
                    ##print(tmp[j[0]]-tmp[group_8_not_anonymized[0][1][0]])
                    diff = abs(tmp[j[0]]-tmp[group_8_not_anonymized[0][8][0]])
                    closest_group = x
    group_8_backup[group_8_not_anonymized[0][0]][group_8_not_anonymized[0][1]][group_8_not_anonymized[0][2]][group_8_not_anonymized[0][3]][group_8_not_anonymized[0][4]][group_8_not_anonymized[0][5]][group_8_not_anonymized[0][6]][group_8_not_anonymized[0][7]][closest_group].extend(group_8_not_anonymized[0][8])
    group_8_backup[group_8_not_anonymized[0][0]][group_8_not_anonymized[0][1]][group_8_not_anonymized[0][2]][group_8_not_anonymized[0][3]][group_8_not_anonymized[0][4]][group_8_not_anonymized[0][5]][group_8_not_anonymized[0][6]][group_8_not_anonymized[0][7]].remove(group_8_not_anonymized[0][8])
    group_8_not_anonymized = []
    for pp,nn in enumerate(group_8_backup):
        for xx,mm in enumerate(nn):
            for yy,ii in enumerate(mm):
                for zz,jj in enumerate(ii):
                    for p,n in enumerate(jj):
                        for x,m in enumerate(n):
                            for y,i in enumerate(m):
                                    for z,j in enumerate(i):
                                        for k in j:
                                            if 0<len(k)<20:
                                                #print(pp,xx,yy,zz,p,x,y,z)
                                                #print(len(k))
                                                #print(k)
                                                group_8_not_anonymized.append([pp,xx,yy,zz,p,x,y,z,k])
    if len(group_8_not_anonymized) != 0:
        print(group_8_not_anonymized[0])
    #print(time.time()-start)
print(time.time()-start_1)

0.0011205673217773438


In [876]:
start = time.time()
data_address_company_anonymized_7 = copy.deepcopy(data_address_company_anonymized_df_6.values)
column = list(data_address_company_anonymized_df_6.columns)
tmp = data_address_company_anonymized_df_6["address_float_8"].values
for dd in group_8_backup:
    for cc in dd:
        for bb in cc:
            for aa in bb:
                for v in aa:
                    for p in v:
                        for q in p:
                            for y in q:
                                for i in y:
                                    a = 0
                                    anonymized_data = []
                                    for x,j in enumerate(i):
                                        if x == 0:
                                            fix = tmp[j]
                                        elif fix != tmp[j]:
                                            a+=1
                                            for k in range(1,64):
                                                if fix[0:-k] == tmp[j][0:-k]:
                                                    anonymized_data.append(fix[0:-k] + "".join("0" for l in range(k)))
                                                    break
                                    if a > 0:
                                        for x,anony in enumerate(anonymized_data):
                                            if x == 0:
                                                fix = anony
                                            elif fix != anony:
                                                for k in range(1,64):
                                                    if fix[0:-k] == anony[0:-k]:
                                                        fix = (fix[0:-k] + "".join("0" for l in range(k)))
                                                        break
                                        column_num = column.index("address_float_8")
                                        for m in i:
                                            data_address_company_anonymized_7[m,column_num] = fix
                                            for o in range(column_num+1,len(column)):
                                                if o%2 == 1:
                                                    data_address_company_anonymized_7[m,o] = "".join("0" for l in range(64))
                                                else:
                                                    data_address_company_anonymized_7[m,o] = np.float64(0)
print(time.time()-start)

1.2083961963653564


In [877]:
data_address_company_anonymized_df_7 = pd.DataFrame(data_address_company_anonymized_7,columns = column)

In [ ]:
Counter(data_address_company_anonymized_df_7.address_float_8.values).most_common()

In [ ]:
address_float_value_anonymized = []
for i in range(9):
    tmp = []
    for j in data_address_company_anonymized_df_7["address_float_{}".format(i)].values:
        tmp.append(float_to_num(j))
    address_float_value_anonymized.append(tmp)

In [ ]:
len(address_float_value_anonymized[0])

In [ ]:
len(address_float_value_anonymized)

In [ ]:
address_int_anonymized = []
for j in range(len(address_float_value_anonymized[0])):
    for x,i in enumerate(address_float_value_anonymized):
        if x == 0:
            tmp = bin(int(i[j]*np.float64(1.7976931348623157e+308)/np.float64(2**48-1))).replace("b","").rjust(48,"0")
        else:
            tmp+=bin(int(i[j]*np.float64(1.7976931348623157e+308)/np.float64(2**48-1))).replace("b","").rjust(48,"0")
            #print(tmp)
    address_int_anonymized.append(tmp)

In [ ]:
address_anonymized = []
for i in address_int_anonymized:
    for j in range(0,len(i),8):
        if j == 0:
            tmp = chr(int(i[j:j+8],2))
        else:
            tmp+=chr(int(i[j:j+8],2))
    address_anonymized.append(tmp[:tmp.find("\x00")])

In [ ]:
data_anonymized_final = pd.DataFrame(data_address_company_anonymized_df_7.address_english.values,columns=["raw"])

In [ ]:
data_anonymized_final["anonymized"] = address_anonymized

In [ ]:
data_anonymized_final

In [ ]:
int(address_int_anonymized[0][0:48],2)

In [ ]:
binary(np.float64(64))

In [ ]:
import mpmath as mp

In [ ]:
mp.dps = 1108

In [ ]:
mp.dps

In [ ]:
bin(2**1148-1)

In [ ]:
address_float_value_2 = []
for i in data_address_company_anonymized_df_7.address_float_2:
    address_float_value_2.append(float_to_num(i))

In [ ]:
np.float64(int(data[j:j+48],2)*(2**48-1)/1.7976931348623157e+308)

In [ ]:
bin(100).replace("b","").rjust(48,"0")